# 🔧 오픈소스 문서 텍스트 추출 테스트 파이프라인

## 📋 개요
이 노트북은 **다양한 문서 형식에서 텍스트를 추출하고 저장하는 종합 테스트 환경**을 제공합니다.

### 🎯 주요 목적
- **다양한 문서 형식 지원**: PDF, DOCX, PPTX, XLSX, TXT, HTML, 이미지 파일
- **추출 방법 비교 테스트**: PyPDF2, pdfplumber, OCR 등 성능 및 정확도 비교
- **OCR 처리 검증**: Tesseract 기반 이미지 PDF 페이지별 처리 및 저장
- **결과 저장 및 검증**: 추출된 텍스트의 품질 검사 및 파일 저장

### 💰 비용 효율성
- **완전 무료**: 모든 라이브러리가 오픈소스 (상용 API 불필요)
- **로컬 처리**: 외부 서비스 의존성 없음, 데이터 보안 보장
- **확장 가능**: 새로운 문서 형식 및 처리 방법 쉽게 추가

### 🔄 백엔드 연동 가능
추출된 텍스트는 백엔드 `tools.py`의 `chunked_texts()` 함수와 연동하여 임베딩 생성 및 벡터스토어 저장이 가능합니다.

### 📊 테스트 시나리오
1. **단일 파일 테스트**: 개별 문서의 텍스트 추출 및 검증
2. **배치 테스트**: 여러 파일 동시 처리 및 성능 측정
3. **방법별 비교**: 같은 파일을 다른 방법으로 처리하여 결과 비교
4. **품질 검증**: 추출된 텍스트의 정확도 및 완성도 평가

### 📁 디렉토리 구조
```
📂 data/
├── 📂 input_docs/           # 테스트할 원본 문서들
├── 📂 output/
│   ├── 📂 extracted_texts/  # 추출된 텍스트 파일들
│   ├── 📂 test_results/     # 테스트 결과 및 비교 분석
│   └── 📂 logs/            # 처리 로그 및 메타데이터
```

---

## 📦 1단계: 오픈소스 라이브러리 로딩 및 환경 설정

In [1]:
# 📦 오픈소스 라이브러리 초고속 확인 (설치 없음)

import subprocess
import sys
import os
import json
import re
import time
from datetime import datetime
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("⚡ 초고속 라이브러리 상태 확인 (설치 제외)")
print("=" * 50)

# 라이브러리 상태 확인만 (설치 안함)
library_status = {}
LIBRARIES_LOADED = True

# 필수 라이브러리 목록
required_libs = {
    'PyPDF2': 'PDF 기본 처리',
    'pdfplumber': 'PDF 고급 처리', 
    'docx': 'Word 문서',
    'pptx': 'PowerPoint',
    'pandas': '데이터 처리',
    'bs4': 'HTML 파싱',
    'html2text': 'HTML 변환',
    'pytesseract': 'OCR',
    'PIL': '이미지 처리',
    'pdf2image': 'PDF→이미지',
    'numpy': '수치 계산',
    'chardet': '인코딩 감지'
}

print("🔍 라이브러리 상태 점검:")
missing_count = 0
success_count = 0

for lib_name, description in required_libs.items():
    try:
        # 안전한 임포트 시도
        if lib_name == 'docx':
            exec("from docx import Document", {})
        elif lib_name == 'pptx':
            exec("from pptx import Presentation", {})
        elif lib_name == 'bs4':
            exec("from bs4 import BeautifulSoup", {})
        elif lib_name == 'PIL':
            exec("from PIL import Image", {})
        else:
            exec(f"import {lib_name}", {})
        
        library_status[lib_name] = True
        print(f"   ✅ {lib_name} ({description})")
        success_count += 1
    except ImportError:
        library_status[lib_name] = False
        print(f"   ❌ {lib_name} ({description}) - 누락")
        missing_count += 1
        LIBRARIES_LOADED = False
    except Exception as e:
        # 기타 예외 처리
        library_status[lib_name] = False
        print(f"   ⚠️ {lib_name} ({description}) - 오류: {str(e)[:50]}")
        missing_count += 1
        LIBRARIES_LOADED = False

# tiktoken 선택적 확인
try:
    exec("import tiktoken", {})
    library_status['tiktoken'] = True
    print(f"   ✅ tiktoken (토큰 계산) - 선택적")
except ImportError:
    library_status['tiktoken'] = False
    print(f"   ⚠️ tiktoken (토큰 계산) - 누락 (선택적)")
except Exception as e:
    library_status['tiktoken'] = False
    print(f"   ⚠️ tiktoken (토큰 계산) - 오류: {str(e)[:50]} (선택적)")

# OCR 환경 확인 (안전하게)
tesseract_found = False
if library_status.get('pytesseract', False):
    try:
        import pytesseract
        tesseract_version = pytesseract.get_tesseract_version()
        print(f"   ✅ Tesseract OCR: {tesseract_version}")
        tesseract_found = True
    except Exception as e:
        print(f"   ⚠️ Tesseract OCR 시스템 설치 필요: {str(e)[:50]}")

print("\n" + "=" * 50)
print("📊 상태 요약:")
print(f"   • 설치된 라이브러리: {success_count}/{len(required_libs)}")
print(f"   • 누락된 라이브러리: {missing_count}개")
print(f"   • Tesseract OCR: {'✅' if tesseract_found else '⚠️'}")

if missing_count > 0:
    print(f"\n💡 누락된 라이브러리 목록:")
    missing_libs = [lib for lib, status in library_status.items() 
                   if not status and lib in required_libs]
    for lib in missing_libs:
        print(f"   - {lib}")
    
    print(f"\n🔧 설치 방법 (필요시 다음 셀 실행):")
    print(f"   다음 셀을 실행하여 자동 설치")

if LIBRARIES_LOADED:
    print(f"\n🎉 모든 필수 라이브러리 준비 완료!")
else:
    print(f"\n⚠️ 일부 라이브러리 누락. 기능이 제한될 수 있습니다.")

print(f"\n⚡ 초고속 확인 완료! (약 3-5초)")

# 안전한 임포트 (에러 방지)
try:
    if library_status.get('PyPDF2', False):
        import PyPDF2
except: pass

try:
    if library_status.get('pandas', False):
        import pandas as pd
except: pass

try:
    if library_status.get('numpy', False):
        import numpy as np
except: pass

try:
    if library_status.get('chardet', False):
        import chardet
except: pass

print(f"\n💡 사용법:")
print(f"   • 다음 셀: 누락 라이브러리 자동 설치")
print(f"   • 모든 체크 완료, 안전하게 다음 단계 진행 가능")

⚡ 초고속 라이브러리 상태 확인 (설치 제외)
🔍 라이브러리 상태 점검:
   ✅ PyPDF2 (PDF 기본 처리)
   ❌ pdfplumber (PDF 고급 처리) - 누락
   ✅ docx (Word 문서)
   ✅ pptx (PowerPoint)
   ✅ pptx (PowerPoint)
   ✅ pandas (데이터 처리)
   ✅ bs4 (HTML 파싱)
   ✅ html2text (HTML 변환)
   ✅ pytesseract (OCR)
   ✅ PIL (이미지 처리)
   ✅ pdf2image (PDF→이미지)
   ✅ numpy (수치 계산)
   ✅ chardet (인코딩 감지)
   ✅ tiktoken (토큰 계산) - 선택적
   ✅ Tesseract OCR: 5.3.0

📊 상태 요약:
   • 설치된 라이브러리: 11/12
   • 누락된 라이브러리: 1개
   • Tesseract OCR: ✅

💡 누락된 라이브러리 목록:
   - pdfplumber

🔧 설치 방법 (필요시 다음 셀 실행):
   다음 셀을 실행하여 자동 설치

⚠️ 일부 라이브러리 누락. 기능이 제한될 수 있습니다.

⚡ 초고속 확인 완료! (약 3-5초)

💡 사용법:
   • 다음 셀: 누락 라이브러리 자동 설치
   • 모든 체크 완료, 안전하게 다음 단계 진행 가능
   ✅ pandas (데이터 처리)
   ✅ bs4 (HTML 파싱)
   ✅ html2text (HTML 변환)
   ✅ pytesseract (OCR)
   ✅ PIL (이미지 처리)
   ✅ pdf2image (PDF→이미지)
   ✅ numpy (수치 계산)
   ✅ chardet (인코딩 감지)
   ✅ tiktoken (토큰 계산) - 선택적
   ✅ Tesseract OCR: 5.3.0

📊 상태 요약:
   • 설치된 라이브러리: 11/12
   • 누락된 라이브러리: 1개
   • Tesseract OCR: ✅

💡 누락된 라이브러리 목록:
   - pdfplumber

🔧 설치 

In [2]:
# 🚀 선택적 라이브러리 설치 (필요시에만 실행)

# 위 셀에서 누락된 라이브러리가 있는 경우에만 이 셀을 실행하세요!

print("🚀 누락된 라이브러리 자동 설치")
print("=" * 40)

# 안전한 설치 함수
def safe_install(package_name, display_name=None):
    """안전한 패키지 설치"""
    display_name = display_name or package_name
    try:
        print(f"   🔄 {display_name} 설치 중...")
        result = subprocess.run(
            [sys.executable, '-m', 'pip', 'install', package_name, '-q'],
            capture_output=True, text=True, timeout=300  # 5분 타임아웃
        )
        if result.returncode == 0:
            print(f"   ✅ {display_name} 설치 완료")
            return True
        else:
            print(f"   ❌ {display_name} 설치 실패: {result.stderr[:100]}")
            return False
    except subprocess.TimeoutExpired:
        print(f"   ⏰ {display_name} 설치 타임아웃 (5분 초과)")
        return False
    except Exception as e:
        print(f"   ❌ {display_name} 설치 오류: {str(e)[:100]}")
        return False

# 이전 셀의 결과 확인
if 'library_status' in globals() and 'required_libs' in globals():
    missing_libs = [lib for lib, status in library_status.items() 
                   if not status and lib in required_libs]
    
    if missing_libs:
        print(f"📦 {len(missing_libs)}개 라이브러리 설치 필요:")
        for lib in missing_libs:
            print(f"   - {lib}")
        
        print(f"\n🔄 설치 시작...")
        
        # 실제 패키지명 매핑
        package_names = {
            'docx': 'python-docx',
            'pptx': 'python-pptx', 
            'bs4': 'beautifulsoup4',
            'PIL': 'Pillow'
        }
        
        success_count = 0
        total_libs = len(missing_libs)
        
        for i, lib in enumerate(missing_libs, 1):
            package_name = package_names.get(lib, lib)
            print(f"\n[{i}/{total_libs}] {lib} 설치:")
            
            if safe_install(package_name, lib):
                success_count += 1
        
        print(f"\n📊 설치 결과: {success_count}/{total_libs} 성공")
        
        if success_count == total_libs:
            print("🎉 모든 라이브러리 설치 완료!")
            print("💡 이제 다음 셀들을 계속 실행하세요.")
        elif success_count > 0:
            print("⚠️ 일부 설치 완료. 실패한 라이브러리는 수동 설치 필요")
        else:
            print("❌ 모든 설치 실패. 네트워크 또는 권한 문제 확인 필요")
            
    else:
        print("✅ 설치할 라이브러리가 없습니다.")
        print("💡 모든 필수 라이브러리가 이미 설치되어 있습니다!")
        
else:
    print("❌ 이전 셀을 먼저 실행하세요.")
    print("💡 라이브러리 상태 확인이 필요합니다.")

print(f"\n⚡ 설치 작업 완료!")

🚀 누락된 라이브러리 자동 설치
📦 1개 라이브러리 설치 필요:
   - pdfplumber

🔄 설치 시작...

[1/1] pdfplumber 설치:
   🔄 pdfplumber 설치 중...
   ✅ pdfplumber 설치 완료

📊 설치 결과: 1/1 성공
🎉 모든 라이브러리 설치 완료!
💡 이제 다음 셀들을 계속 실행하세요.

⚡ 설치 작업 완료!
   ✅ pdfplumber 설치 완료

📊 설치 결과: 1/1 성공
🎉 모든 라이브러리 설치 완료!
💡 이제 다음 셀들을 계속 실행하세요.

⚡ 설치 작업 완료!


## ⚙️ 2단계: 파일 형식 지원 및 설정 관리

In [3]:
# ⚙️ 문서 텍스트 추출 테스트 환경 설정

import logging
from datetime import datetime

class DocumentExtractionTestConfig:
    """문서 텍스트 추출 테스트 전용 설정 클래스"""
    
    def __init__(self, base_dir=None):
        try:
            # 기본 디렉토리 설정
            if base_dir is None:
                base_dir = Path("/home/admin/wkms-aws/jupyter_notebook/data")
            else:
                base_dir = Path(base_dir)
            
            # 디렉토리 구조 정의
            self.base_dir = base_dir
            self.input_docs_dir = base_dir / "input_docs"
            self.output_dir = base_dir / "output"
            self.extracted_texts_dir = self.output_dir / "extracted_texts"
            self.test_results_dir = self.output_dir / "test_results"
            self.logs_dir = self.output_dir / "logs"
            
            # 필요한 디렉토리 생성 (안전하게)
            self._create_directories()
            
            # 지원하는 파일 형식 정의
            self.supported_formats = {
                'pdf': ['.pdf'],
                'word': ['.docx', '.doc'],
                'powerpoint': ['.pptx', '.ppt'],
                'excel': ['.xlsx', '.xls'],
                'text': ['.txt', '.md', '.rtf'],
                'html': ['.html', '.htm', '.xml'],
                'image': ['.png', '.jpg', '.jpeg', '.tiff', '.bmp']
            }
            
            # 추출 방법 설정
            self.extraction_methods = {
                'pdf': ['pypdf2', 'pdfplumber', 'ocr'],
                'word': ['python-docx'],
                'powerpoint': ['python-pptx'],
                'excel': ['openpyxl'],
                'text': ['builtin'],
                'html': ['beautifulsoup'],
                'image': ['tesseract']
            }
            
            # OCR 설정
            self.ocr_config = {
                'enabled': True,
                'language': 'kor+eng',  # 한국어 + 영어
                'dpi': 300,
                'save_page_files': True,  # 페이지별 개별 파일 저장
                'save_combined_file': True  # 전체 통합 파일 저장
            }
            
            # 테스트 설정
            self.test_config = {
                'compare_methods': True,  # 여러 방법 비교 테스트
                'measure_performance': True,  # 성능 측정
                'validate_results': True,  # 결과 검증
                'save_detailed_logs': True,  # 상세 로그 저장
                'create_summary_report': True  # 요약 보고서 생성
            }
            
            # 로거 설정 (안전하게)
            self.logger = self._setup_logger()
            
            # 토크나이저는 지연 로딩 (실제 필요할 때만 초기화)
            self._tokenizer = None
            
            print("⚙️ 문서 텍스트 추출 테스트 환경 설정 완료!")
            self._print_basic_config()
            
        except Exception as e:
            print(f"❌ 설정 초기화 오류: {e}")
            print("💡 기본값으로 초기화를 시도합니다...")
            self._init_fallback()
    
    def _init_fallback(self):
        """초기화 실패시 기본값으로 설정"""
        try:
            self.base_dir = Path.cwd() / "data"
            self.input_docs_dir = self.base_dir / "input_docs"
            self.output_dir = self.base_dir / "output"
            self.extracted_texts_dir = self.output_dir / "extracted_texts"
            self.test_results_dir = self.output_dir / "test_results"
            self.logs_dir = self.output_dir / "logs"
            self.logger = None
            self._tokenizer = None
            print("⚠️ 기본값으로 초기화됨")
        except Exception as e:
            print(f"❌ 기본값 초기화도 실패: {e}")
    
    @property
    def tokenizer(self):
        """토크나이저를 지연 로딩으로 초기화합니다."""
        if self._tokenizer is None:
            try:
                print("🔄 토크나이저 초기화 중...")
                import tiktoken
                self._tokenizer = tiktoken.get_encoding("cl100k_base")  # GPT-4 기본 토크나이저
                print("✅ 토크나이저 초기화 완료!")
            except Exception as e:
                print(f"⚠️ 토크나이저 초기화 실패: {e}")
                self._tokenizer = None
        return self._tokenizer
    
    def _create_directories(self):
        """필요한 디렉토리들을 안전하게 생성합니다."""
        directories = [
            self.input_docs_dir,
            self.output_dir,
            self.extracted_texts_dir,
            self.test_results_dir,
            self.logs_dir
        ]
        
        for directory in directories:
            try:
                directory.mkdir(parents=True, exist_ok=True)
            except Exception as e:
                print(f"⚠️ 디렉토리 생성 실패 {directory}: {e}")
    
    def _setup_logger(self):
        """로거를 안전하게 설정합니다."""
        try:
            logger = logging.getLogger('document_extraction_test')
            logger.setLevel(logging.INFO)
            
            # 기존 핸들러 제거 (중복 방지)
            logger.handlers.clear()
            
            # 파일 핸들러
            log_file = self.logs_dir / f"extraction_test_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
            file_handler = logging.FileHandler(log_file, encoding='utf-8')
            file_handler.setLevel(logging.INFO)
            
            # 포맷터
            formatter = logging.Formatter(
                '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
            )
            file_handler.setFormatter(formatter)
            logger.addHandler(file_handler)
            
            return logger
            
        except Exception as e:
            print(f"⚠️ 로거 설정 실패: {e}")
            return None
    
    def get_file_type(self, file_path):
        """파일 확장자를 기반으로 파일 타입을 판별합니다."""
        try:
            file_path = Path(file_path)
            extension = file_path.suffix.lower()
            
            for file_type, extensions in self.supported_formats.items():
                if extension in extensions:
                    return file_type
            return 'unknown'
        except Exception:
            return 'unknown'
    
    def is_supported_file(self, file_path):
        """파일이 지원되는 형식인지 확인합니다."""
        return self.get_file_type(file_path) != 'unknown'
    
    def get_available_methods(self, file_type):
        """파일 타입에 대해 사용 가능한 추출 방법들을 반환합니다."""
        return self.extraction_methods.get(file_type, [])
    
    def _print_basic_config(self):
        """기본 설정만 출력합니다 (무거운 작업 제외)."""
        try:
            print("=" * 70)
            print("📁 디렉토리 구조:")
            print(f"   📂 입력: {self.input_docs_dir}")
            print(f"   📂 출력: {self.output_dir}")
            print(f"   📝 텍스트: {self.extracted_texts_dir}")
            print(f"   📊 테스트 결과: {self.test_results_dir}")
            print(f"   📋 로그: {self.logs_dir}")
            
            print(f"\n📋 지원 파일 형식:")
            for file_type, extensions in self.supported_formats.items():
                methods = ', '.join(self.get_available_methods(file_type))
                print(f"   {file_type.upper()}: {', '.join(extensions)} ({methods})")
            
            print(f"\n🔍 OCR 설정:")
            print(f"   언어: {self.ocr_config['language']}")
            print(f"   DPI: {self.ocr_config['dpi']}")
            print(f"   페이지별 저장: {self.ocr_config['save_page_files']}")
            
            print(f"\n🧪 테스트 설정:")
            for key, value in self.test_config.items():
                print(f"   {key}: {value}")
            
            print("✅ 기본 설정 완료! (토크나이저는 필요시 자동 로드)")
        except Exception as e:
            print(f"⚠️ 설정 출력 오류: {e}")
    
    def print_file_summary(self):
        """입력 디렉토리의 파일 현황을 안전하게 출력합니다."""
        try:
            print("\n📄 입력 파일 현황 스캔 중...")
            
            if not self.input_docs_dir.exists():
                print(f"❌ 입력 디렉토리가 없습니다: {self.input_docs_dir}")
                return
            
            files = list(self.input_docs_dir.glob("*"))
            supported_files = [f for f in files if f.is_file() and self.is_supported_file(f)]
            
            print(f"📊 파일 현황:")
            print(f"   전체 파일: {len([f for f in files if f.is_file()])}개")
            print(f"   지원 파일: {len(supported_files)}개")
            
            if supported_files:
                print(f"   📋 지원 파일 목록:")
                for file_path in supported_files[:10]:  # 최대 10개만 표시
                    try:
                        file_type = self.get_file_type(file_path)
                        file_size = file_path.stat().st_size / 1024 / 1024  # MB
                        print(f"      • {file_path.name} ({file_type}, {file_size:.2f}MB)")
                    except Exception as e:
                        print(f"      • {file_path.name} (정보 읽기 실패)")
                
                if len(supported_files) > 10:
                    print(f"      ... 및 {len(supported_files) - 10}개 더")
        except Exception as e:
            print(f"❌ 파일 스캔 오류: {e}")

# 테스트 환경 설정 인스턴스 생성 (안전한 초기화)
try:
    config = DocumentExtractionTestConfig()
    EXTRACTION_CONFIG = config
    print("\n💡 사용법:")
    print("   • config.print_file_summary(): 입력 파일 현황 확인")
    print("   • config.tokenizer: 토크나이저 (필요시 자동 로드)")
    print("   • 모든 무거운 작업은 지연 로딩으로 최적화됨")
except Exception as e:
    print(f"❌ 설정 생성 실패: {e}")
    print("💡 다음 셀을 실행하기 전에 문제를 해결하세요.")

⚙️ 문서 텍스트 추출 테스트 환경 설정 완료!
📁 디렉토리 구조:
   📂 입력: /home/admin/wkms-aws/jupyter_notebook/data/input_docs
   📂 출력: /home/admin/wkms-aws/jupyter_notebook/data/output
   📝 텍스트: /home/admin/wkms-aws/jupyter_notebook/data/output/extracted_texts
   📊 테스트 결과: /home/admin/wkms-aws/jupyter_notebook/data/output/test_results
   📋 로그: /home/admin/wkms-aws/jupyter_notebook/data/output/logs

📋 지원 파일 형식:
   PDF: .pdf (pypdf2, pdfplumber, ocr)
   WORD: .docx, .doc (python-docx)
   POWERPOINT: .pptx, .ppt (python-pptx)
   EXCEL: .xlsx, .xls (openpyxl)
   TEXT: .txt, .md, .rtf (builtin)
   HTML: .html, .htm, .xml (beautifulsoup)
   IMAGE: .png, .jpg, .jpeg, .tiff, .bmp (tesseract)

🔍 OCR 설정:
   언어: kor+eng
   DPI: 300
   페이지별 저장: True

🧪 테스트 설정:
   compare_methods: True
   measure_performance: True
   validate_results: True
   save_detailed_logs: True
   create_summary_report: True
✅ 기본 설정 완료! (토크나이저는 필요시 자동 로드)

💡 사용법:
   • config.print_file_summary(): 입력 파일 현황 확인
   • config.tokenizer: 토크나이저 (필요시 자동 로드)
 

## 📄 3단계: 고속 문서 로딩 및 파일 정보 추출

In [4]:
# 📄 문서 로더 및 파일 관리 클래스 (안전한 버전)

class DocumentTestLoader:
    """테스트를 위한 문서 로딩 및 파일 관리 클래스 (개선된 안전성)"""
    
    def __init__(self, config):
        try:
            self.config = config
            self.logger = config.logger if hasattr(config, 'logger') else None
            self.loaded_documents = {}  # 로드된 문서 캐시
            print("📄 문서 로더 초기화 완료!")
        except Exception as e:
            print(f"⚠️ 문서 로더 초기화 오류: {e}")
            self.config = None
            self.logger = None
            self.loaded_documents = {}
        
    def scan_input_directory(self, show_details=True, timeout_seconds=30):
        """입력 디렉토리를 안전하게 스캔하여 지원되는 문서들을 찾습니다."""
        
        if not self.config:
            print("❌ 설정이 초기화되지 않았습니다.")
            return []
        
        try:
            if not self.config.input_docs_dir.exists():
                print(f"❌ 입력 디렉토리가 없습니다: {self.config.input_docs_dir}")
                return []
            
            print(f"📂 입력 디렉토리 스캔: {self.config.input_docs_dir}")
            print("=" * 60)
            
            # 타임아웃을 고려한 파일 스캔
            import time
            start_time = time.time()
            
            all_files = []
            for file_path in self.config.input_docs_dir.glob("*"):
                if time.time() - start_time > timeout_seconds:
                    print(f"⏰ 스캔 타임아웃 ({timeout_seconds}초) - 부분 결과 반환")
                    break
                all_files.append(file_path)
            
            supported_files = []
            file_types = {}
            
            for file_path in all_files:
                try:
                    if file_path.is_file():
                        file_type = self.config.get_file_type(file_path)
                        
                        if file_type != 'unknown':
                            supported_files.append(file_path)
                            
                            if file_type not in file_types:
                                file_types[file_type] = []
                            file_types[file_type].append(file_path)
                except Exception as e:
                    if self.logger:
                        self.logger.warning(f"파일 처리 오류 {file_path}: {e}")
                    continue
            
            # 결과 요약
            print(f"📊 스캔 결과:")
            print(f"   전체 파일: {len([f for f in all_files if f.is_file()])}개")
            print(f"   지원 파일: {len(supported_files)}개")
            print(f"   지원 타입: {len(file_types)}개")
            
            # 타입별 상세 정보
            if show_details and file_types:
                print(f"\n📋 파일 타입별 분류:")
                for file_type, files in file_types.items():
                    try:
                        methods = ', '.join(self.config.get_available_methods(file_type))
                        print(f"\n   📁 {file_type.upper()} ({len(files)}개) - 추출방법: {methods}")
                        
                        for file_path in files[:5]:  # 최대 5개만 표시
                            try:
                                file_size = file_path.stat().st_size / 1024 / 1024  # MB
                                print(f"      • {file_path.name} ({file_size:.2f}MB)")
                            except Exception:
                                print(f"      • {file_path.name} (크기 정보 없음)")
                        
                        if len(files) > 5:
                            print(f"      ... 및 {len(files) - 5}개 더")
                    except Exception as e:
                        print(f"      ⚠️ {file_type} 정보 처리 오류: {e}")
            
            return supported_files
            
        except Exception as e:
            print(f"❌ 디렉토리 스캔 오류: {e}")
            return []
    
    def load_document_info(self, file_path):
        """문서의 기본 정보를 안전하게 로드합니다."""
        try:
            file_path = Path(file_path)
            
            if not file_path.exists():
                raise FileNotFoundError(f"파일이 없습니다: {file_path}")
            
            file_type = self.config.get_file_type(file_path)
            if file_type == 'unknown':
                raise ValueError(f"지원하지 않는 파일 형식: {file_path.suffix}")
            
            # 파일 기본 정보
            file_stat = file_path.stat()
            
            doc_info = {
                'file_path': str(file_path),
                'file_name': file_path.name,
                'file_stem': file_path.stem,
                'file_type': file_type,
                'file_size_bytes': file_stat.st_size,
                'file_size_mb': file_stat.st_size / 1024 / 1024,
                'modified_time': datetime.fromtimestamp(file_stat.st_mtime),
                'available_methods': self.config.get_available_methods(file_type),
                'load_time': datetime.now()
            }
            
            # 캐시에 저장
            self.loaded_documents[str(file_path)] = doc_info
            
            return doc_info
            
        except Exception as e:
            print(f"❌ 문서 정보 로드 오류 {file_path}: {e}")
            return None
    
    def get_test_files_by_type(self, file_type=None, max_files=None):
        """파일 타입별로 테스트 파일들을 안전하게 가져옵니다."""
        try:
            supported_files = self.scan_input_directory(show_details=False)
            
            if file_type:
                filtered_files = [f for f in supported_files if self.config.get_file_type(f) == file_type]
            else:
                filtered_files = supported_files
            
            if max_files and max_files > 0:
                filtered_files = filtered_files[:max_files]
            
            return filtered_files
        except Exception as e:
            print(f"❌ 파일 필터링 오류: {e}")
            return []
    
    def create_test_batch(self, include_types=None, exclude_large_files=True, size_limit_mb=50):
        """테스트를 위한 파일 배치를 안전하게 생성합니다."""
        
        try:
            print(f"📦 테스트 배치 생성")
            print("=" * 50)
            
            all_files = self.scan_input_directory(show_details=False)
            test_batch = []
            
            for file_path in all_files:
                try:
                    file_type = self.config.get_file_type(file_path)
                    
                    # 타입 필터링
                    if include_types and file_type not in include_types:
                        continue
                    
                    # 크기 필터링
                    if exclude_large_files:
                        try:
                            file_size_mb = file_path.stat().st_size / 1024 / 1024
                            if file_size_mb > size_limit_mb:
                                continue
                        except Exception:
                            continue  # 크기를 읽을 수 없으면 제외
                    
                    doc_info = self.load_document_info(file_path)
                    if doc_info:
                        test_batch.append(doc_info)
                        
                except Exception as e:
                    if self.logger:
                        self.logger.warning(f"배치 생성 중 파일 처리 오류 {file_path}: {e}")
                    continue
            
            # 배치 요약
            print(f"📊 배치 구성:")
            print(f"   선택된 파일: {len(test_batch)}개")
            
            if test_batch:
                # 타입별 통계
                type_counts = {}
                total_size = 0
                
                for doc in test_batch:
                    try:
                        file_type = doc['file_type']
                        if file_type not in type_counts:
                            type_counts[file_type] = 0
                        type_counts[file_type] += 1
                        total_size += doc.get('file_size_mb', 0)
                    except Exception:
                        continue
                
                print(f"   총 크기: {total_size:.2f}MB")
                print(f"   타입별 분포:")
                for file_type, count in type_counts.items():
                    print(f"      • {file_type}: {count}개")
            
            return test_batch
            
        except Exception as e:
            print(f"❌ 테스트 배치 생성 오류: {e}")
            return []
    
    def save_document_list(self, documents, filename="document_list.json"):
        """문서 목록을 안전하게 JSON 파일로 저장합니다."""
        try:
            if not self.config:
                print("❌ 설정이 초기화되지 않았습니다.")
                return None
                
            output_file = self.config.test_results_dir / filename
            
            # datetime 객체를 문자열로 변환
            serializable_docs = []
            for doc in documents:
                try:
                    serializable_doc = doc.copy()
                    for key, value in serializable_doc.items():
                        if isinstance(value, datetime):
                            serializable_doc[key] = value.isoformat()
                        elif isinstance(value, Path):
                            serializable_doc[key] = str(value)
                    serializable_docs.append(serializable_doc)
                except Exception as e:
                    print(f"⚠️ 문서 직렬화 오류: {e}")
                    continue
            
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(serializable_docs, f, ensure_ascii=False, indent=2)
            
            print(f"💾 문서 목록 저장: {output_file}")
            return output_file
            
        except Exception as e:
            print(f"❌ 문서 목록 저장 오류: {e}")
            return None

# 안전한 문서 로더 인스턴스 생성
try:
    if 'config' in globals() and config:
        document_loader = DocumentTestLoader(config)
        print("\n💡 사용법:")
        print("   • document_loader.scan_input_directory(): 입력 디렉토리 스캔")
        print("   • document_loader.create_test_batch(): 테스트 배치 생성")
        print("   • document_loader.get_test_files_by_type('pdf'): PDF 파일만 가져오기")
        print("   • config.print_file_summary(): 입력 파일 현황 확인")
    else:
        print("❌ 이전 셀의 config가 필요합니다. 이전 셀을 먼저 실행하세요.")
except Exception as e:
    print(f"❌ 문서 로더 생성 오류: {e}")
    print("💡 이전 셀을 다시 실행한 후 이 셀을 실행하세요.")

📄 문서 로더 초기화 완료!

💡 사용법:
   • document_loader.scan_input_directory(): 입력 디렉토리 스캔
   • document_loader.create_test_batch(): 테스트 배치 생성
   • document_loader.get_test_files_by_type('pdf'): PDF 파일만 가져오기
   • config.print_file_summary(): 입력 파일 현황 확인


## 🔤 4단계: 텍스트 추출 엔진 구현 (다중 방식 지원)

In [5]:
# 🔤 텍스트 추출 테스트 엔진 (개선된 안전성)

import time
import json
from pathlib import Path

class DocumentTextExtractor:
    """다양한 문서 형식에서 텍스트를 추출하는 테스트 엔진 (안전한 버전)"""
    
    def __init__(self, config):
        try:
            self.config = config
            self.logger = config.logger if hasattr(config, 'logger') else None
            self.extraction_stats = {}  # 추출 통계
            
            # 라이브러리 가용성 체크
            self._check_library_availability()
            
            print("🔤 문서 텍스트 추출 엔진 초기화 완료!")
            
        except Exception as e:
            print(f"❌ 텍스트 추출 엔진 초기화 오류: {e}")
            self.config = None
            self.logger = None
            self.extraction_stats = {}
    
    def _check_library_availability(self):
        """필요한 라이브러리들의 가용성을 확인합니다."""
        self.available_libs = {}
        
        # 라이브러리 체크 (안전하게)
        libs_to_check = [
            ('PyPDF2', 'PyPDF2'),
            ('pdfplumber', 'pdfplumber'),
            ('python-docx', 'docx'),
            ('python-pptx', 'pptx'),
            ('openpyxl', 'openpyxl'),
            ('beautifulsoup4', 'bs4'),
            ('pytesseract', 'pytesseract'),
            ('Pillow', 'PIL'),
            ('pdf2image', 'pdf2image')
        ]
        
        for package_name, import_name in libs_to_check:
            try:
                exec(f"import {import_name}", {})
                self.available_libs[package_name] = True
            except ImportError:
                self.available_libs[package_name] = False
            except Exception:
                self.available_libs[package_name] = False
    
    def extract_text(self, file_path, method='auto', save_result=True, timeout_seconds=300):
        """
        파일에서 텍스트를 안전하게 추출합니다.
        
        Args:
            file_path: 파일 경로
            method: 추출 방법 ('auto', 'pypdf2', 'pdfplumber', 'ocr' 등)
            save_result: 결과를 파일로 저장할지 여부
            timeout_seconds: 타임아웃 (기본 5분)
        """
        
        if not self.config:
            return {'success': False, 'error': '설정이 초기화되지 않았습니다.'}
        
        try:
            file_path = Path(file_path)
            
            if not file_path.exists():
                return {'success': False, 'error': f'파일이 존재하지 않습니다: {file_path}'}
            
            file_type = self.config.get_file_type(file_path)
            
            print(f"🔄 텍스트 추출 시작: {file_path.name}")
            print(f"   📄 파일 타입: {file_type}")
            print(f"   🔧 추출 방법: {method}")
            
            extraction_result = {
                'file_path': str(file_path),
                'file_name': file_path.name,
                'file_type': file_type,
                'extraction_method': method,
                'start_time': time.time(),
                'success': False,
                'text': '',
                'text_length': 0,
                'metadata': {},
                'error': None,
                'saved_files': []
            }
            
            # 타임아웃 체크를 위한 시작 시간
            start_time = time.time()
            
            # 파일 타입별 추출 실행
            if file_type == 'pdf':
                result = self._extract_from_pdf(file_path, method, timeout_seconds)
            elif file_type == 'word':
                result = self._extract_from_docx(file_path)
            elif file_type == 'powerpoint':
                result = self._extract_from_pptx(file_path)
            elif file_type == 'excel':
                result = self._extract_from_excel(file_path)
            elif file_type == 'text':
                result = self._extract_from_text(file_path)
            elif file_type == 'html':
                result = self._extract_from_html(file_path)
            elif file_type == 'image':
                result = self._extract_from_image(file_path)
            else:
                raise ValueError(f"지원하지 않는 파일 형식: {file_type}")
            
            # 타임아웃 체크
            if time.time() - start_time > timeout_seconds:
                raise TimeoutError(f"추출 시간 초과 ({timeout_seconds}초)")
            
            # 결과 업데이트
            if result and isinstance(result, dict):
                extraction_result.update(result)
                extraction_result['success'] = True
                extraction_result['text_length'] = len(result.get('text', ''))
                
                # 결과 저장 (옵션)
                if save_result and result.get('text'):
                    try:
                        saved_file = self._save_extracted_text(
                            result['text'], 
                            file_path.stem, 
                            result.get('extraction_method', method),
                            result.get('metadata', {})
                        )
                        if saved_file:
                            extraction_result['saved_files'].append(str(saved_file))
                    except Exception as save_error:
                        print(f"⚠️ 파일 저장 실패: {save_error}")
                
                print(f"   ✅ 추출 성공: {extraction_result['text_length']:,}자")
            else:
                raise ValueError("추출 결과가 올바르지 않습니다.")
                
        except Exception as e:
            extraction_result['error'] = str(e)
            print(f"   ❌ 추출 실패: {e}")
            if self.logger:
                self.logger.error(f"텍스트 추출 실패 {file_path}: {e}")
        
        # 처리 시간 계산
        extraction_result['processing_time'] = time.time() - extraction_result['start_time']
        print(f"   ⏱️ 처리 시간: {extraction_result['processing_time']:.2f}초")
        
        # 통계 업데이트
        self._update_stats(extraction_result)
        
        return extraction_result
    
    def _extract_from_pdf(self, file_path, method='auto', timeout_seconds=300):
        """PDF에서 텍스트 추출 (안전한 버전)"""
        
        if method == 'auto':
            # PyPDF2 먼저 시도
            if self.available_libs.get('PyPDF2', False):
                try:
                    result = self._extract_pdf_pypdf2(file_path)
                    if len(result.get('text', '').strip()) > 50:  # 충분한 텍스트가 추출된 경우
                        return result
                    else:
                        print(f"   💡 PyPDF2로 충분한 텍스트 추출 안됨")
                except Exception as e:
                    print(f"   💡 PyPDF2 실패: {e}")
            
            # OCR 시도
            if self.available_libs.get('pytesseract', False) and self.available_libs.get('pdf2image', False):
                try:
                    print(f"   💡 OCR 시도...")
                    return self._extract_pdf_ocr(file_path)
                except Exception as e:
                    print(f"   💡 OCR 실패: {e}")
            
            # 기본 텍스트 읽기 시도
            try:
                return self._extract_from_text(file_path)
            except:
                raise ValueError("사용 가능한 PDF 추출 방법이 없습니다.")
        
        elif method == 'pypdf2':
            if not self.available_libs.get('PyPDF2', False):
                raise ValueError("PyPDF2 라이브러리가 설치되지 않았습니다.")
            return self._extract_pdf_pypdf2(file_path)
            
        elif method == 'pdfplumber':
            if not self.available_libs.get('pdfplumber', False):
                raise ValueError("pdfplumber 라이브러리가 설치되지 않았습니다.")
            return self._extract_pdf_pdfplumber(file_path)
            
        elif method == 'ocr':
            if not (self.available_libs.get('pytesseract', False) and self.available_libs.get('pdf2image', False)):
                raise ValueError("OCR 관련 라이브러리가 설치되지 않았습니다.")
            return self._extract_pdf_ocr(file_path)
        else:
            raise ValueError(f"알 수 없는 PDF 추출 방법: {method}")
    
    def _extract_pdf_pypdf2(self, file_path):
        """PyPDF2를 사용한 PDF 텍스트 추출 (안전한 버전)"""
        print(f"   🔄 PyPDF2 방법 시도...")
        start_time = time.time()
        
        try:
            import PyPDF2
            
            with open(file_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                
                metadata = {
                    'pages': len(pdf_reader.pages),
                    'extraction_method': 'pypdf2',
                    'processing_time': 0
                }
                
                # PDF 메타데이터 추출 (안전하게)
                try:
                    if pdf_reader.metadata:
                        metadata.update({
                            'title': str(pdf_reader.metadata.get('/Title', '')),
                            'author': str(pdf_reader.metadata.get('/Author', '')),
                            'creator': str(pdf_reader.metadata.get('/Creator', '')),
                        })
                except Exception:
                    pass  # 메타데이터 추출 실패시 무시
                
                # 텍스트 추출
                text_parts = []
                for page_num, page in enumerate(pdf_reader.pages, 1):
                    try:
                        page_text = page.extract_text()
                        if page_text and page_text.strip():
                            text_parts.append(f"=== 페이지 {page_num} ===\n{page_text}\n")
                    except Exception as e:
                        text_parts.append(f"=== 페이지 {page_num} (추출 실패) ===\n")
                        if self.logger:
                            self.logger.warning(f"페이지 {page_num} 추출 실패: {e}")
                
                full_text = '\n'.join(text_parts)
                metadata['processing_time'] = time.time() - start_time
                
                print(f"   ✅ PyPDF2 성공: {len(full_text):,}자, {metadata['processing_time']:.2f}초")
                
                return {
                    'text': full_text,
                    'metadata': metadata,
                    'extraction_method': 'pypdf2'
                }
                
        except Exception as e:
            raise Exception(f"PyPDF2 추출 오류: {e}")
    
    def _extract_pdf_ocr(self, file_path):
        """OCR을 사용한 PDF 텍스트 추출 (기본 구현)"""
        print(f"   🔄 OCR 방법 시도...")
        
        try:
            # 기본적인 OCR 구현 (실제로는 더 복잡한 로직 필요)
            import pytesseract
            from pdf2image import convert_from_path
            
            # PDF를 이미지로 변환 (첫 페이지만)
            images = convert_from_path(file_path, dpi=150, first_page=1, last_page=1)
            
            if images:
                text = pytesseract.image_to_string(images[0], lang='kor+eng')
                
                return {
                    'text': f"=== OCR 추출 결과 ===\n{text}",
                    'metadata': {
                        'pages': 1,
                        'extraction_method': 'ocr',
                        'language': 'kor+eng'
                    },
                    'extraction_method': 'ocr'
                }
            else:
                raise ValueError("PDF를 이미지로 변환할 수 없습니다.")
                
        except Exception as e:
            raise Exception(f"OCR 추출 오류: {e}")
    
    def _extract_from_text(self, file_path):
        """텍스트 파일에서 텍스트 추출"""
        print(f"   🔄 텍스트 파일 읽기...")
        
        try:
            # 인코딩 자동 감지
            encodings = ['utf-8', 'cp949', 'euc-kr', 'latin-1']
            
            for encoding in encodings:
                try:
                    with open(file_path, 'r', encoding=encoding) as file:
                        text = file.read()
                        
                    return {
                        'text': text,
                        'metadata': {
                            'file_size': file_path.stat().st_size,
                            'encoding': encoding,
                            'extraction_method': 'builtin'
                        },
                        'extraction_method': 'builtin'
                    }
                except UnicodeDecodeError:
                    continue
            
            raise ValueError("지원되는 인코딩으로 파일을 읽을 수 없습니다.")
            
        except Exception as e:
            raise Exception(f"텍스트 파일 읽기 오류: {e}")
    
    def _extract_from_docx(self, file_path):
        """Word 문서에서 텍스트 추출"""
        if not self.available_libs.get('python-docx', False):
            raise ValueError("python-docx 라이브러리가 설치되지 않았습니다.")
        
        print(f"   🔄 Word 문서 처리...")
        
        try:
            from docx import Document
            
            doc = Document(file_path)
            text_parts = []
            
            for paragraph in doc.paragraphs:
                if paragraph.text.strip():
                    text_parts.append(paragraph.text)
            
            full_text = '\n'.join(text_parts)
            
            return {
                'text': full_text,
                'metadata': {
                    'paragraphs': len(doc.paragraphs),
                    'extraction_method': 'python-docx'
                },
                'extraction_method': 'python-docx'
            }
            
        except Exception as e:
            raise Exception(f"Word 문서 처리 오류: {e}")
    
    def _extract_from_pptx(self, file_path):
        """PowerPoint에서 텍스트 추출"""
        if not self.available_libs.get('python-pptx', False):
            raise ValueError("python-pptx 라이브러리가 설치되지 않았습니다.")
        
        print(f"   🔄 PowerPoint 처리...")
        
        try:
            from pptx import Presentation
            
            prs = Presentation(file_path)
            text_parts = []
            
            for slide_num, slide in enumerate(prs.slides, 1):
                slide_texts = []
                for shape in slide.shapes:
                    if hasattr(shape, 'text') and shape.text.strip():
                        slide_texts.append(shape.text)
                
                if slide_texts:
                    text_parts.append(f"=== 슬라이드 {slide_num} ===\n" + '\n'.join(slide_texts))
            
            full_text = '\n\n'.join(text_parts)
            
            return {
                'text': full_text,
                'metadata': {
                    'slides': len(prs.slides),
                    'extraction_method': 'python-pptx'
                },
                'extraction_method': 'python-pptx'
            }
            
        except Exception as e:
            raise Exception(f"PowerPoint 처리 오류: {e}")
    
    def _extract_from_excel(self, file_path):
        """Excel에서 텍스트 추출 (기본 구현)"""
        raise NotImplementedError("Excel 추출은 아직 구현되지 않았습니다.")
    
    def _extract_from_html(self, file_path):
        """HTML에서 텍스트 추출 (기본 구현)"""
        raise NotImplementedError("HTML 추출은 아직 구현되지 않았습니다.")
    
    def _extract_from_image(self, file_path):
        """이미지에서 텍스트 추출 (기본 구현)"""
        raise NotImplementedError("이미지 추출은 아직 구현되지 않았습니다.")
    
    def _save_extracted_text(self, text, filename, method, metadata=None):
        """추출된 텍스트를 파일로 저장"""
        try:
            if not self.config or not hasattr(self.config, 'extracted_texts_dir'):
                return None
                
            output_file = self.config.extracted_texts_dir / f"{filename}_{method}.txt"
            
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(f"# 추출 방법: {method}\n")
                f.write(f"# 추출 시간: {datetime.now().isoformat()}\n")
                if metadata:
                    f.write(f"# 메타데이터: {json.dumps(metadata, ensure_ascii=False, indent=2)}\n")
                f.write("\n" + "="*50 + "\n\n")
                f.write(text)
            
            return output_file
            
        except Exception as e:
            print(f"⚠️ 텍스트 저장 실패: {e}")
            return None
    
    def _update_stats(self, result):
        """추출 통계 업데이트"""
        try:
            file_type = result.get('file_type', 'unknown')
            if file_type not in self.extraction_stats:
                self.extraction_stats[file_type] = {
                    'total_files': 0,
                    'successful_extractions': 0,
                    'failed_extractions': 0,
                    'total_text_length': 0,
                    'total_processing_time': 0.0
                }
            
            stats = self.extraction_stats[file_type]
            stats['total_files'] += 1
            
            if result.get('success', False):
                stats['successful_extractions'] += 1
                stats['total_text_length'] += result.get('text_length', 0)
            else:
                stats['failed_extractions'] += 1
            
            stats['total_processing_time'] += result.get('processing_time', 0.0)
            
        except Exception as e:
            if self.logger:
                self.logger.warning(f"통계 업데이트 실패: {e}")

# 안전한 텍스트 추출 엔진 생성
try:
    if 'config' in globals() and config:
        text_extractor = DocumentTextExtractor(config)
        print("\n💡 사용법:")
        print("   • text_extractor.extract_text('파일경로'): 텍스트 추출")
        print("   • text_extractor.extract_text('파일경로', method='pypdf2'): 특정 방법 사용")
        print("   • text_extractor.extraction_stats: 추출 통계 확인")
    else:
        print("❌ 이전 셀의 config가 필요합니다. 이전 셀을 먼저 실행하세요.")
except Exception as e:
    print(f"❌ 텍스트 추출 엔진 생성 오류: {e}")
    print("💡 이전 셀들을 순서대로 실행한 후 이 셀을 실행하세요.")

🔤 문서 텍스트 추출 엔진 초기화 완료!

💡 사용법:
   • text_extractor.extract_text('파일경로'): 텍스트 추출
   • text_extractor.extract_text('파일경로', method='pypdf2'): 특정 방법 사용
   • text_extractor.extraction_stats: 추출 통계 확인


In [6]:
# 🔤 개별 문서 형식별 추출 메소드 구현 (경량/안전 모드)

# 즉시 피드백을 위해 시작 메시지 출력 (버퍼링 방지)
import sys, time, re
print("⚙️ 메소드 등록 시작…", flush=True)
start_register_ts = time.time()

# 각 메소드는 필요한 라이브러리를 내부에서 임포트하여
# 셀 실행 시 무거운 임포트로 인해 출력이 지연되지 않도록 합니다.

def _extract_pdf_pdfplumber(self, file_path):
    """pdfplumber를 사용한 고급 PDF 텍스트 추출"""
    print(f"   🔄 pdfplumber 방법 시도...", flush=True)
    start_time = time.time()
    
    try:
        import pdfplumber
        
        with pdfplumber.open(file_path) as pdf:
            metadata = {
                'pages': len(pdf.pages),
                'extraction_method': 'pdfplumber',
                'processing_time': 0
            }
            
            # PDF 메타데이터
            try:
                if getattr(pdf, 'metadata', None):
                    metadata.update(pdf.metadata)
            except Exception:
                pass
            
            text_parts = []
            
            for page_num, page in enumerate(pdf.pages, 1):
                try:
                    # 일반 텍스트 추출
                    page_text = page.extract_text() or ""
                    
                    # 테이블 추출
                    try:
                        tables = page.extract_tables()
                    except Exception:
                        tables = None
                    if tables:
                        table_text = f"\n\n=== 페이지 {page_num} 테이블 ===\n"
                        for table_num, table in enumerate(tables, 1):
                            table_text += f"\n--- 테이블 {table_num} ---\n"
                            for row in table:
                                if row:
                                    clean_row = [str(cell).strip() if cell else "" for cell in row]
                                    table_text += " | ".join(clean_row) + "\n"
                        page_text += table_text
                    
                    if page_text.strip():
                        text_parts.append(f"=== 페이지 {page_num} ===\n{page_text}\n")
                
                except Exception as e:
                    text_parts.append(f"=== 페이지 {page_num} (추출 실패: {e}) ===\n")
            
            full_text = '\n'.join(text_parts)
            metadata['processing_time'] = time.time() - start_time
            
            print(f"   ✅ pdfplumber 성공: {len(full_text):,}자, {metadata['processing_time']:.2f}초", flush=True)
            
            return {
                'text': full_text,
                'metadata': metadata,
                'extraction_method': 'pdfplumber'
            }
    
    except ImportError:
        raise ImportError("pdfplumber 라이브러리가 설치되지 않았습니다.")


def _extract_from_docx(self, file_path):
    """Word 문서에서 텍스트 추출"""
    print(f"   🔄 python-docx 방법 시도...", flush=True)
    start_time = time.time()
    
    try:
        from docx import Document
    except Exception as e:
        raise ImportError(f"python-docx 임포트 실패: {e}")

    doc = Document(file_path)
    
    # 메타데이터 추출
    core_props = doc.core_properties
    metadata = {
        'extraction_method': 'python-docx',
        'title': getattr(core_props, 'title', '') or '',
        'author': getattr(core_props, 'author', '') or '',
        'created': str(core_props.created) if getattr(core_props, 'created', None) else '',
        'modified': str(core_props.modified) if getattr(core_props, 'modified', None) else '',
        'paragraphs': len(doc.paragraphs),
        'tables': len(doc.tables),
        'processing_time': 0
    }
    
    # 텍스트 추출
    text_parts = []
    
    # 단락 텍스트
    for para in doc.paragraphs:
        if para.text.strip():
            text_parts.append(para.text)
    
    # 테이블 텍스트
    for table_num, table in enumerate(doc.tables, 1):
        table_text = f"\n\n=== 테이블 {table_num} ===\n"
        for row in table.rows:
            row_cells = [cell.text.strip() for cell in row.cells]
            table_text += " | ".join(row_cells) + "\n"
        text_parts.append(table_text)
    
    full_text = '\n'.join(text_parts)
    metadata['processing_time'] = time.time() - start_time
    
    print(f"   ✅ Word 추출 성공: {len(full_text):,}자, {metadata['processing_time']:.2f}초", flush=True)
    
    return {
        'text': full_text,
        'metadata': metadata,
        'extraction_method': 'python-docx'
    }


def _extract_from_pptx(self, file_path):
    """PowerPoint에서 텍스트 추출"""
    print(f"   🔄 python-pptx 방법 시도...", flush=True)
    start_time = time.time()
    
    try:
        from pptx import Presentation
    except Exception as e:
        raise ImportError(f"python-pptx 임포트 실패: {e}")

    prs = Presentation(file_path)
    
    metadata = {
        'extraction_method': 'python-pptx',
        'slides': len(prs.slides),
        'processing_time': 0
    }
    
    text_parts = []
    
    for slide_num, slide in enumerate(prs.slides, 1):
        slide_text = f"\n\n=== 슬라이드 {slide_num} ===\n"
        
        for shape in slide.shapes:
            if hasattr(shape, "text") and shape.text.strip():
                slide_text += shape.text + "\n"
        
        text_parts.append(slide_text)
    
    full_text = '\n'.join(text_parts)
    metadata['processing_time'] = time.time() - start_time
    
    print(f"   ✅ PowerPoint 추출 성공: {len(full_text):,}자, {metadata['processing_time']:.2f}초", flush=True)
    
    return {
        'text': full_text,
        'metadata': metadata,
        'extraction_method': 'python-pptx'
    }


def _extract_from_excel(self, file_path):
    """Excel에서 텍스트 추출"""
    print(f"   🔄 openpyxl+pandas 방법 시도...", flush=True)
    start_time = time.time()
    
    try:
        import pandas as pd
    except Exception as e:
        raise ImportError(f"pandas 임포트 실패: {e}")
    
    try:
        # 모든 시트 읽기
        excel_data = pd.read_excel(file_path, sheet_name=None)
        
        metadata = {
            'extraction_method': 'openpyxl+pandas',
            'sheets': list(excel_data.keys()),
            'sheet_count': len(excel_data),
            'processing_time': 0
        }
        
        text_parts = []
        
        for sheet_name, df in excel_data.items():
            sheet_text = f"\n\n=== 시트: {sheet_name} ===\n"
            try:
                sheet_text += df.to_string(index=False)
            except Exception:
                sheet_text += str(df)
            text_parts.append(sheet_text)
        
        full_text = '\n'.join(text_parts)
        metadata['processing_time'] = time.time() - start_time
        
        print(f"   ✅ Excel 추출 성공: {len(full_text):,}자, {metadata['processing_time']:.2f}초", flush=True)
        
        return {
            'text': full_text,
            'metadata': metadata,
            'extraction_method': 'openpyxl+pandas'
        }
        
    except Exception as e:
        raise RuntimeError(f"Excel 파일 읽기 실패: {e}")


def _extract_from_text(self, file_path):
    """텍스트 파일에서 내용 추출"""
    print(f"   🔄 텍스트 파일 읽기...", flush=True)
    start_time = time.time()
    
    try:
        import chardet
    except Exception as e:
        raise ImportError(f"chardet 임포트 실패: {e}")

    # 인코딩 감지
    with open(file_path, 'rb') as f:
        raw_data = f.read()
        encoding_result = chardet.detect(raw_data)
        encoding = encoding_result.get('encoding') or 'utf-8'
    
    # 텍스트 읽기
    try:
        with open(file_path, 'r', encoding=encoding) as f:
            text_content = f.read()
    except UnicodeDecodeError:
        # 인코딩 실패시 utf-8로 재시도
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            text_content = f.read()
    
    metadata = {
        'extraction_method': 'text_file',
        'lines': len(text_content.split('\n')),
        'detected_encoding': encoding,
        'confidence': encoding_result.get('confidence', 0),
        'processing_time': time.time() - start_time
    }
    
    print(f"   ✅ 텍스트 추출 성공: {len(text_content):,}자, {metadata['processing_time']:.2f}초", flush=True)
    
    return {
        'text': text_content,
        'metadata': metadata,
        'extraction_method': 'text_file'
    }


def _extract_from_html(self, file_path):
    """HTML에서 텍스트 추출"""
    print(f"   🔄 BeautifulSoup 방법 시도...", flush=True)
    start_time = time.time()
    
    # HTML 파일 읽기
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        html_content = f.read()
    
    try:
        from bs4 import BeautifulSoup
    except Exception as e:
        raise ImportError(f"beautifulsoup4 임포트 실패: {e}")
    
    # BeautifulSoup으로 정제된 텍스트 추출
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # 불필요한 태그 제거
    for tag in soup(['script', 'style', 'meta', 'link', 'head']):
        try:
            tag.decompose()
        except Exception:
            pass
    
    # 테이블 처리
    tables = soup.find_all('table')
    for table in tables:
        table_text = "\n\n=== 테이블 ===\n"
        rows = table.find_all('tr')
        for row in rows:
            cells = row.find_all(['td', 'th'])
            row_text = []
            for cell in cells:
                cell_text = cell.get_text().strip()
                row_text.append(cell_text)
            if row_text:
                table_text += " | ".join(row_text) + "\n"
        table.replace_with(table_text)
    
    # 텍스트 추출 및 정제
    text = soup.get_text()
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)  # HTML 주석
    text = re.sub(r'\n{3,}', '\n\n', text)  # 연속 개행
    text = re.sub(r'[ \t]+', ' ', text)  # 연속 공백
    processed_text = text.strip()
    
    metadata = {
        'extraction_method': 'beautifulsoup',
        'content_length': len(html_content),
        'processed_length': len(processed_text),
        'processing_time': time.time() - start_time
    }
    
    print(f"   ✅ HTML 추출 성공: {len(processed_text):,}자, {metadata['processing_time']:.2f}초", flush=True)
    
    return {
        'text': processed_text,
        'metadata': metadata,
        'extraction_method': 'beautifulsoup'
    }


def _extract_from_image(self, file_path):
    """이미지에서 OCR로 텍스트 추출"""
    print(f"   🔄 이미지 OCR 시도...", flush=True)
    start_time = time.time()
    
    if not self.config.ocr_config['enabled']:
        raise ValueError("OCR이 비활성화되어 있습니다.")
    
    try:
        from PIL import Image
    except Exception as e:
        raise ImportError(f"Pillow 임포트 실패: {e}")
    
    try:
        import pytesseract
    except Exception as e:
        raise ImportError(f"pytesseract 임포트 실패: {e}")

    try:
        image = Image.open(file_path)
        
        # OCR 실행
        text = pytesseract.image_to_string(
            image,
            lang=self.config.ocr_config['language']
        )
        
        metadata = {
            'extraction_method': 'tesseract_ocr',
            'image_size': getattr(image, 'size', None),
            'image_mode': getattr(image, 'mode', None),
            'ocr_language': self.config.ocr_config['language'],
            'processing_time': time.time() - start_time
        }
        
        print(f"   ✅ 이미지 OCR 성공: {len(text):,}자, {metadata['processing_time']:.2f}초", flush=True)
        
        return {
            'text': text,
            'metadata': metadata,
            'extraction_method': 'tesseract_ocr'
        }
        
    except Exception as e:
        raise RuntimeError(f"이미지 OCR 실패: {e}")

# TextExtractor에 모든 메소드 추가
DocumentTextExtractor._extract_pdf_pdfplumber = _extract_pdf_pdfplumber
DocumentTextExtractor._extract_from_docx = _extract_from_docx
DocumentTextExtractor._extract_from_pptx = _extract_from_pptx
DocumentTextExtractor._extract_from_excel = _extract_from_excel
DocumentTextExtractor._extract_from_text = _extract_from_text
DocumentTextExtractor._extract_from_html = _extract_from_html
DocumentTextExtractor._extract_from_image = _extract_from_image

print("📄 모든 문서 형식별 추출 메소드 추가 완료!", flush=True)
print("=" * 50, flush=True)
print("📋 지원하는 추출 방법:", flush=True)
print("   • PDF: PyPDF2, pdfplumber, OCR", flush=True)
print("   • Word: python-docx", flush=True)
print("   • PowerPoint: python-pptx", flush=True)
print("   • Excel: openpyxl + pandas", flush=True)
print("   • HTML: BeautifulSoup", flush=True)
print("   • 이미지: Tesseract OCR", flush=True)
print("   • 텍스트: 인코딩 자동 감지", flush=True)
print(f"⏱️ 등록 완료: {(time.time() - start_register_ts):.2f}초", flush=True)

⚙️ 메소드 등록 시작…
📄 모든 문서 형식별 추출 메소드 추가 완료!
📋 지원하는 추출 방법:
   • PDF: PyPDF2, pdfplumber, OCR
   • Word: python-docx
   • PowerPoint: python-pptx
📄 모든 문서 형식별 추출 메소드 추가 완료!
📋 지원하는 추출 방법:
   • PDF: PyPDF2, pdfplumber, OCR
   • Word: python-docx
   • PowerPoint: python-pptx
   • Excel: openpyxl + pandas
   • HTML: BeautifulSoup
   • 이미지: Tesseract OCR
   • 텍스트: 인코딩 자동 감지
⏱️ 등록 완료: 0.01초
   • Excel: openpyxl + pandas
   • HTML: BeautifulSoup
   • 이미지: Tesseract OCR
   • 텍스트: 인코딩 자동 감지
⏱️ 등록 완료: 0.01초


## 🧪 테스트 실행 및 결과 분석

In [7]:
# 🧪 단일 파일 테스트 함수

def test_single_file(file_path, methods=None, save_results=True):
    """
    단일 파일에 대해 다양한 방법으로 텍스트 추출을 테스트합니다.
    
    Args:
        file_path: 테스트할 파일 경로
        methods: 테스트할 추출 방법들 (None이면 자동 선택)
        save_results: 결과를 파일로 저장할지 여부
    """
    file_path = Path(file_path)
    
    if not file_path.exists():
        print(f"❌ 파일이 없습니다: {file_path}")
        return None
    
    file_type = config.get_file_type(file_path)
    if file_type == 'unknown':
        print(f"❌ 지원하지 않는 파일 형식: {file_path.suffix}")
        return None
    
    print(f"🧪 단일 파일 테스트: {file_path.name}")
    print("=" * 60)
    print(f"📄 파일 타입: {file_type}")
    print(f"📊 파일 크기: {file_path.stat().st_size / 1024 / 1024:.2f}MB")
    
    # 테스트할 방법들 결정
    if methods is None:
        methods = config.get_available_methods(file_type)
        # PDF의 경우 모든 방법 테스트
        if file_type == 'pdf':
            methods = ['pypdf2', 'pdfplumber', 'ocr']
    
    print(f"🔧 테스트 방법: {', '.join(methods)}")
    print()
    
    # 각 방법별로 테스트 실행
    test_results = []
    
    for method in methods:
        print(f"🔄 {method} 테스트 시작")
        print("-" * 30)
        
        try:
            result = text_extractor.extract_text(
                file_path, 
                method=method, 
                save_result=save_results
            )
            test_results.append(result)
            
            # 결과 요약
            if result['success']:
                text_length = result['text_length']
                processing_time = result['processing_time']
                print(f"✅ 성공: {text_length:,}자, {processing_time:.2f}초")
                
                # 텍스트 미리보기
                if text_length > 0:
                    preview = result['text'][:200].replace('\n', ' ').strip()
                    print(f"👀 미리보기: {preview}...")
                    
                    # 한글 비율 계산
                    korean_chars = len([c for c in result['text'] if '가' <= c <= '힣'])
                    if text_length > 0:
                        korean_ratio = korean_chars / text_length * 100
                        print(f"🇰🇷 한글 비율: {korean_ratio:.1f}%")
            else:
                print(f"❌ 실패: {result['error']}")
        
        except Exception as e:
            print(f"❌ 예외 발생: {e}")
            test_results.append({
                'file_path': str(file_path),
                'file_type': file_type,
                'extraction_method': method,
                'success': False,
                'error': str(e),
                'text_length': 0,
                'processing_time': 0
            })
        
        print()
    
    # 결과 비교 분석
    if len(test_results) > 1:
        print("📊 방법별 결과 비교")
        print("=" * 40)
        
        successful_results = [r for r in test_results if r['success']]
        
        if successful_results:
            # 최고 품질 (가장 많은 텍스트)
            best_quality = max(successful_results, key=lambda x: x['text_length'])
            print(f"🎯 최고 품질: {best_quality['extraction_method']} ({best_quality['text_length']:,}자)")
            
            # 최고 속도 (가장 빠른 처리)
            best_speed = min(successful_results, key=lambda x: x['processing_time'])
            print(f"⚡ 최고 속도: {best_speed['extraction_method']} ({best_speed['processing_time']:.2f}초)")
            
            # 추천 방법
            if best_quality['text_length'] > 1000:
                print(f"💡 추천: {best_quality['extraction_method']} (충분한 텍스트 추출)")
            elif file_type == 'pdf' and any(r['extraction_method'] == 'ocr' for r in successful_results):
                ocr_result = next(r for r in successful_results if r['extraction_method'] == 'ocr')
                print(f"💡 추천: OCR (이미지 기반 PDF로 판단)")
            else:
                print(f"💡 추천: {best_speed['extraction_method']} (속도 우선)")
        else:
            print("❌ 모든 방법이 실패했습니다.")
    
    # 결과 저장
    if save_results:
        result_file = config.test_results_dir / f"{file_path.stem}_test_results.json"
        
        # datetime 객체를 문자열로 변환
        serializable_results = []
        for result in test_results:
            serializable_result = result.copy()
            for key, value in serializable_result.items():
                if isinstance(value, datetime):
                    serializable_result[key] = value.isoformat()
                elif isinstance(value, Path):
                    serializable_result[key] = str(value)
            serializable_results.append(serializable_result)
        
        with open(result_file, 'w', encoding='utf-8') as f:
            json.dump({
                'file_info': {
                    'name': file_path.name,
                    'path': str(file_path),
                    'type': file_type,
                    'size_mb': file_path.stat().st_size / 1024 / 1024
                },
                'test_config': {
                    'methods_tested': methods,
                    'timestamp': datetime.now().isoformat()
                },
                'results': serializable_results
            }, f, ensure_ascii=False, indent=2)
        
        print(f"💾 테스트 결과 저장: {result_file.name}")
    
    return test_results

# 테스트 편의 함수들
def quick_test_pdf(file_path):
    """PDF 파일 빠른 테스트 (모든 방법)"""
    return test_single_file(file_path, methods=['pypdf2', 'pdfplumber', 'ocr'])

def quick_test_auto(file_path):
    """자동 방법 선택으로 빠른 테스트"""
    return test_single_file(file_path, methods=['auto'])

print("🧪 단일 파일 테스트 함수 정의 완료!")
print()
print("💡 사용법:")
print("   • test_single_file('파일경로'): 모든 방법으로 테스트")
print("   • quick_test_pdf('pdf파일'): PDF 전용 테스트")
print("   • quick_test_auto('파일경로'): 자동 방법 선택")

🧪 단일 파일 테스트 함수 정의 완료!

💡 사용법:
   • test_single_file('파일경로'): 모든 방법으로 테스트
   • quick_test_pdf('pdf파일'): PDF 전용 테스트
   • quick_test_auto('파일경로'): 자동 방법 선택


In [8]:
# 📊 배치 테스트 및 성능 분석 함수

def test_multiple_files(file_paths=None, file_types=None, max_files=10, save_results=True):
    """
    여러 파일에 대해 배치 테스트를 수행합니다.
    
    Args:
        file_paths: 특정 파일들의 경로 리스트
        file_types: 테스트할 파일 타입들 ('pdf', 'word' 등)
        max_files: 최대 테스트 파일 수
        save_results: 결과를 파일로 저장할지 여부
    """
    print("📊 배치 테스트 시작")
    print("=" * 50)
    
    # 테스트할 파일들 수집
    if file_paths:
        test_files = [Path(f) for f in file_paths if Path(f).exists()]
    else:
        # 입력 디렉토리에서 파일 수집
        all_files = document_loader.scan_input_directory(show_details=False)
        
        if file_types:
            test_files = [f for f in all_files if config.get_file_type(f) in file_types]
        else:
            test_files = all_files
        
        # 파일 수 제한
        if max_files and len(test_files) > max_files:
            test_files = test_files[:max_files]
    
    if not test_files:
        print("❌ 테스트할 파일이 없습니다.")
        return []
    
    print(f"📁 테스트 대상: {len(test_files)}개 파일")
    
    # 파일별 테스트 실행
    all_results = []
    failed_files = []
    
    for i, file_path in enumerate(test_files, 1):
        print(f"\n🔄 파일 {i}/{len(test_files)}: {file_path.name}")
        print("-" * 40)
        
        try:
            # 자동 방법으로 테스트 (빠른 처리)
            result = text_extractor.extract_text(file_path, method='auto', save_result=save_results)
            all_results.append(result)
            
            if result['success']:
                print(f"✅ 성공: {result['text_length']:,}자, {result['processing_time']:.2f}초")
            else:
                print(f"❌ 실패: {result['error']}")
                failed_files.append(file_path.name)
        
        except Exception as e:
            print(f"❌ 예외: {e}")
            failed_files.append(file_path.name)
    
    # 전체 결과 분석
    print(f"\n📊 배치 테스트 결과 요약")
    print("=" * 50)
    
    successful_results = [r for r in all_results if r['success']]
    
    print(f"📈 성공률: {len(successful_results)}/{len(all_results)} ({len(successful_results)/len(all_results)*100:.1f}%)")
    
    if successful_results:
        # 처리 통계
        total_text = sum(r['text_length'] for r in successful_results)
        total_time = sum(r['processing_time'] for r in successful_results)
        avg_time = total_time / len(successful_results)
        
        print(f"📝 총 추출 텍스트: {total_text:,}자")
        print(f"⏱️ 총 처리 시간: {total_time:.2f}초")
        print(f"📊 평균 처리 시간: {avg_time:.2f}초/파일")
        
        # 파일 타입별 통계
        type_stats = {}
        for result in successful_results:
            file_type = result['file_type']
            if file_type not in type_stats:
                type_stats[file_type] = {'count': 0, 'total_text': 0, 'total_time': 0}
            
            type_stats[file_type]['count'] += 1
            type_stats[file_type]['total_text'] += result['text_length']
            type_stats[file_type]['total_time'] += result['processing_time']
        
        print(f"\n📋 파일 타입별 통계:")
        for file_type, stats in type_stats.items():
            avg_text = stats['total_text'] / stats['count']
            avg_time = stats['total_time'] / stats['count']
            print(f"   {file_type.upper()}: {stats['count']}개, 평균 {avg_text:,.0f}자, {avg_time:.2f}초")
    
    if failed_files:
        print(f"\n❌ 실패한 파일들:")
        for filename in failed_files:
            print(f"   • {filename}")
    
    # 결과 저장
    if save_results and all_results:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        result_file = config.test_results_dir / f"batch_test_results_{timestamp}.json"
        
        # 저장용 데이터 준비
        batch_summary = {
            'test_info': {
                'timestamp': datetime.now().isoformat(),
                'total_files': len(test_files),
                'successful_files': len(successful_results),
                'failed_files': len(failed_files),
                'success_rate': len(successful_results) / len(all_results) * 100 if all_results else 0
            },
            'performance_summary': {
                'total_text_length': sum(r['text_length'] for r in successful_results),
                'total_processing_time': sum(r['processing_time'] for r in successful_results),
                'average_processing_time': avg_time if successful_results else 0
            } if successful_results else {},
            'type_statistics': type_stats if successful_results else {},
            'failed_files': failed_files,
            'detailed_results': all_results
        }
        
        with open(result_file, 'w', encoding='utf-8') as f:
            json.dump(batch_summary, f, ensure_ascii=False, indent=2, default=str)
        
        print(f"\n💾 배치 테스트 결과 저장: {result_file.name}")
    
    return all_results

def analyze_extraction_performance():
    """추출기의 성능 통계를 분석합니다."""
    stats = text_extractor.get_extraction_stats()
    
    if not stats:
        print("📊 아직 추출 통계가 없습니다. 먼저 테스트를 실행하세요.")
        return
    
    print("📊 텍스트 추출 성능 분석")
    print("=" * 50)
    
    total_files = 0
    total_success = 0
    total_time = 0
    
    for file_type, methods in stats.items():
        print(f"\n📁 {file_type.upper()} 파일:")
        
        for method, method_stats in methods.items():
            count = method_stats['count']
            success = method_stats['success']
            avg_time = method_stats['total_time'] / count if count > 0 else 0
            avg_text = method_stats['total_text_length'] / success if success > 0 else 0
            success_rate = success / count * 100 if count > 0 else 0
            
            print(f"   🔧 {method}:")
            print(f"      • 시도: {count}회")
            print(f"      • 성공: {success}회 ({success_rate:.1f}%)")
            print(f"      • 평균 시간: {avg_time:.2f}초")
            print(f"      • 평균 텍스트: {avg_text:,.0f}자")
            
            total_files += count
            total_success += success
            total_time += method_stats['total_time']
    
    print(f"\n🎯 전체 요약:")
    print(f"   📊 총 처리: {total_files}회")
    print(f"   ✅ 성공률: {total_success}/{total_files} ({total_success/total_files*100:.1f}%)")
    print(f"   ⏱️ 총 시간: {total_time:.2f}초")
    print(f"   📈 평균 시간: {total_time/total_files:.2f}초/파일" if total_files > 0 else "")

print("📊 배치 테스트 및 성능 분석 함수 정의 완료!")
print()
print("💡 사용법:")
print("   • test_multiple_files(): 모든 파일 배치 테스트")
print("   • test_multiple_files(file_types=['pdf']): PDF만 테스트")
print("   • analyze_extraction_performance(): 성능 통계 분석")

📊 배치 테스트 및 성능 분석 함수 정의 완료!

💡 사용법:
   • test_multiple_files(): 모든 파일 배치 테스트
   • test_multiple_files(file_types=['pdf']): PDF만 테스트
   • analyze_extraction_performance(): 성능 통계 분석


## 🚀 실제 테스트 실행 예제

In [9]:
# 🔍 입력 디렉토리 스캔 및 파일 현황 확인

print("📂 입력 디렉토리 스캔 및 테스트 파일 현황")
print("=" * 60)

# 입력 디렉토리 스캔
available_files = document_loader.scan_input_directory(show_details=True)

print(f"\n📋 테스트 가능한 파일 현황:")
if available_files:
    # 파일 타입별 분류
    files_by_type = {}
    for file_path in available_files:
        file_type = config.get_file_type(file_path)
        if file_type not in files_by_type:
            files_by_type[file_type] = []
        files_by_type[file_type].append(file_path)
    
    print(f"   총 {len(available_files)}개 파일")
    print(f"   지원 타입: {len(files_by_type)}개")
    
    # 타입별 추천 테스트 파일
    for file_type, files in files_by_type.items():
        print(f"\n   📁 {file_type.upper()} ({len(files)}개):")
        
        # 크기가 적당한 파일들 우선 추천 (테스트용)
        recommended_files = []
        for file_path in files:
            size_mb = file_path.stat().st_size / 1024 / 1024
            if size_mb < 10:  # 10MB 미만 추천
                recommended_files.append((file_path, size_mb))
        
        # 크기순 정렬
        recommended_files.sort(key=lambda x: x[1])
        
        for file_path, size_mb in recommended_files[:3]:  # 상위 3개만 표시
            print(f"      • {file_path.name} ({size_mb:.2f}MB) ⭐ 추천")
        
        if len(files) > len(recommended_files):
            large_files = len(files) - len(recommended_files)
            print(f"      • ... 및 {large_files}개 큰 파일들 (10MB+)")

else:
    print("   ❌ 테스트할 파일이 없습니다.")
    print(f"   💡 다음 디렉토리에 문서를 추가하세요: {config.input_docs_dir}")

print(f"\n🎯 다음 단계:")
print(f"   1. 단일 파일 테스트: test_single_file('파일경로')")
print(f"   2. 배치 테스트: test_multiple_files()")
print(f"   3. PDF 전용 테스트: quick_test_pdf('pdf파일경로')")
print(f"   4. 성능 분석: analyze_extraction_performance()")

# 전역 변수로 저장 (다음 셀에서 사용)
AVAILABLE_TEST_FILES = available_files

📂 입력 디렉토리 스캔 및 테스트 파일 현황
📂 입력 디렉토리 스캔: /home/admin/wkms-aws/jupyter_notebook/data/input_docs
📊 스캔 결과:
   전체 파일: 3개
   지원 파일: 3개
   지원 타입: 1개

📋 파일 타입별 분류:

   📁 PDF (3개) - 추출방법: pypdf2, pdfplumber, ocr
      • 20_ProductSpec_SmartInsulinPump_KO_v0.1.pdf (0.32MB)
      • test1.pdf (1.26MB)
      • test.pdf (11.85MB)

📋 테스트 가능한 파일 현황:
   총 3개 파일
   지원 타입: 1개

   📁 PDF (3개):
      • 20_ProductSpec_SmartInsulinPump_KO_v0.1.pdf (0.32MB) ⭐ 추천
      • test1.pdf (1.26MB) ⭐ 추천
      • ... 및 1개 큰 파일들 (10MB+)

🎯 다음 단계:
   1. 단일 파일 테스트: test_single_file('파일경로')
   2. 배치 테스트: test_multiple_files()
   3. PDF 전용 테스트: quick_test_pdf('pdf파일경로')
   4. 성능 분석: analyze_extraction_performance()


In [10]:
# 🎯 특정 파일로 테스트 실행 (예제)

# 사용 가능한 파일이 있는 경우 자동으로 테스트 실행
if 'AVAILABLE_TEST_FILES' in globals() and AVAILABLE_TEST_FILES:
    print("🧪 자동 테스트 실행 (첫 번째 파일)")
    print("=" * 50)
    
    # 첫 번째 파일로 테스트
    test_file = AVAILABLE_TEST_FILES[0]
    print(f"📄 테스트 파일: {test_file.name}")
    
    # 파일 타입에 따른 테스트
    file_type = config.get_file_type(test_file)
    
    if file_type == 'pdf':
        print("📄 PDF 파일 - 모든 추출 방법 테스트")
        results = quick_test_pdf(test_file)
    else:
        print(f"📄 {file_type.upper()} 파일 - 자동 방법 테스트")
        results = quick_test_auto(test_file)
    
    print(f"\n✅ 테스트 완료!")
    
else:
    print("💡 수동 테스트 예제")
    print("=" * 30)
    print("# 특정 파일 테스트 예제:")
    print("# test_file_path = '/home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf'")
    print("# results = test_single_file(test_file_path)")
    print()
    print("# PDF 파일 전용 테스트:")
    print("# results = quick_test_pdf(test_file_path)")
    print()
    print("# 여러 파일 배치 테스트:")
    print("# batch_results = test_multiple_files(max_files=5)")
    print()
    print("# PDF만 배치 테스트:")
    print("# pdf_results = test_multiple_files(file_types=['pdf'], max_files=3)")

# 수동 테스트용 편의 함수
def run_example_tests():
    """예제 테스트들을 실행합니다."""
    if not AVAILABLE_TEST_FILES:
        print("❌ 테스트할 파일이 없습니다.")
        return
    
    print("🧪 예제 테스트 실행")
    print("=" * 40)
    
    # 파일 타입별로 하나씩 테스트
    tested_types = set()
    
    for file_path in AVAILABLE_TEST_FILES[:5]:  # 최대 5개
        file_type = config.get_file_type(file_path)
        
        if file_type not in tested_types:
            print(f"\n📄 {file_type.upper()} 테스트: {file_path.name}")
            print("-" * 30)
            
            try:
                result = text_extractor.extract_text(file_path, method='auto')
                
                if result['success']:
                    print(f"✅ 성공: {result['text_length']:,}자, {result['processing_time']:.2f}초")
                else:
                    print(f"❌ 실패: {result['error']}")
                    
            except Exception as e:
                print(f"❌ 예외: {e}")
            
            tested_types.add(file_type)
    
    print(f"\n🎯 {len(tested_types)}개 파일 타입 테스트 완료!")

print("\n💡 추가 테스트 함수:")
print("   • run_example_tests(): 파일 타입별 예제 테스트")
print("   • analyze_extraction_performance(): 성능 통계 확인")

🧪 자동 테스트 실행 (첫 번째 파일)
📄 테스트 파일: 20_ProductSpec_SmartInsulinPump_KO_v0.1.pdf
📄 PDF 파일 - 모든 추출 방법 테스트
🧪 단일 파일 테스트: 20_ProductSpec_SmartInsulinPump_KO_v0.1.pdf
📄 파일 타입: pdf
📊 파일 크기: 0.32MB
🔧 테스트 방법: pypdf2, pdfplumber, ocr

🔄 pypdf2 테스트 시작
------------------------------
🔄 텍스트 추출 시작: 20_ProductSpec_SmartInsulinPump_KO_v0.1.pdf
   📄 파일 타입: pdf
   🔧 추출 방법: pypdf2
   🔄 PyPDF2 방법 시도...
   ✅ PyPDF2 성공: 1,616자, 0.14초
   ✅ 추출 성공: 1,616자
   ⏱️ 처리 시간: 0.14초
✅ 성공: 1,616자, 0.14초
👀 미리보기: === 페이지 1 ===   스마트 인슐린 펌프   제품 사양서 (데모용)  문서번호: PROD-INS-PUMP-001 버전: 0.1 작성일: 2025-08-12 ※     본문서는데모용가상자료이며 ,        규제제출또는진료의사결정에사용할수없습니다 .  === 페이지 2 ===  그림1.   사용시나리오예시(병원/가정/운동/ 시스템개요)  === 페이...
🇰🇷 한글 비율: 44.7%

🔄 pdfplumber 테스트 시작
------------------------------
🔄 텍스트 추출 시작: 20_ProductSpec_SmartInsulinPump_KO_v0.1.pdf
   📄 파일 타입: pdf
   🔧 추출 방법: pdfplumber
   ❌ 추출 실패: pdfplumber 라이브러리가 설치되지 않았습니다.
   ⏱️ 처리 시간: 0.00초
❌ 실패: pdfplumber 라이브러리가 설치되지 않았습니다.

🔄 ocr 테스트 시작
------------------------------
🔄 텍스트 추출 시작: 

## 📖 사용 가이드 및 요약

### 🚀 빠른 시작 가이드

1. **환경 설정**: 첫 번째 셀부터 순서대로 실행
2. **파일 준비**: `/home/admin/wkms-aws/jupyter_notebook/data/input_docs/` 에 테스트할 문서 추가
3. **스캔 실행**: 입력 디렉토리 스캔 셀 실행으로 파일 현황 확인
4. **테스트 실행**: 원하는 테스트 함수 실행

### 🔧 주요 함수 정리

#### 단일 파일 테스트
```python
# 모든 방법으로 테스트
test_single_file('파일경로')

# PDF 전용 테스트 (PyPDF2, pdfplumber, OCR)
quick_test_pdf('pdf파일경로')

# 자동 방법 선택
quick_test_auto('파일경로')
```

#### 배치 테스트
```python
# 모든 파일 테스트 (최대 10개)
test_multiple_files()

# 특정 타입만 테스트
test_multiple_files(file_types=['pdf'])

# 파일 수 제한
test_multiple_files(max_files=5)
```

#### 분석 및 통계
```python
# 성능 통계 확인
analyze_extraction_performance()

# 예제 테스트 실행
run_example_tests()
```

### 📊 지원하는 파일 형식 및 방법

| 파일 형식 | 확장자 | 추출 방법 | 특징 |
|-----------|--------|-----------|------|
| **PDF** | .pdf | PyPDF2, pdfplumber, OCR | 텍스트/이미지 기반 자동 감지 |
| **Word** | .docx, .doc | python-docx | 텍스트, 테이블 추출 |
| **PowerPoint** | .pptx, .ppt | python-pptx | 슬라이드별 텍스트 |
| **Excel** | .xlsx, .xls | openpyxl+pandas | 시트별 데이터 |
| **HTML** | .html, .htm, .xml | BeautifulSoup | 태그 제거, 테이블 변환 |
| **텍스트** | .txt, .md, .rtf | builtin | 인코딩 자동 감지 |
| **이미지** | .png, .jpg, .jpeg, .tiff, .bmp | Tesseract OCR | 한영 문자 인식 |

### 📁 출력 파일 구조

```
📂 data/output/
├── 📂 extracted_texts/      # 추출된 텍스트 파일들
│   ├── filename_method_timestamp.txt
│   └── filename_page_001.txt (OCR 페이지별)
├── 📂 test_results/         # 테스트 결과 JSON
│   ├── filename_test_results.json
│   └── batch_test_results_timestamp.json
└── 📂 logs/                 # 처리 로그
    └── extraction_test_timestamp.log
```

### 💡 성능 최적화 팁

1. **작은 파일부터**: 큰 파일은 처리 시간이 오래 걸림
2. **PDF 방법 선택**: 
   - 텍스트 기반 → PyPDF2 (빠름)
   - 레이아웃 복잡 → pdfplumber
   - 이미지 기반 → OCR (느림)
3. **배치 크기 조절**: `max_files` 매개변수로 처리량 조절
4. **결과 저장**: `save_results=False`로 저장 생략 가능

### 🔧 문제 해결

- **라이브러리 오류**: 첫 번째 셀 재실행으로 누락 라이브러리 설치
- **Tesseract 오류**: 시스템에 Tesseract OCR 설치 필요
- **메모리 부족**: 큰 파일이나 많은 파일 처리 시 배치 크기 줄이기
- **인코딩 오류**: 텍스트 파일의 인코딩 자동 감지로 해결

---

In [11]:
# 🎉 오픈소스 문서 텍스트 추출 테스트 환경 완료!

print("🎉 오픈소스 문서 텍스트 추출 테스트 환경 구성 완료!")
print("=" * 60)
print()

# 환경 상태 확인
print("🔧 환경 상태:")
print(f"   • 라이브러리 로딩: {'✅' if LIBRARIES_LOADED else '❌'}")
print(f"   • Tesseract OCR: {'✅' if tesseract_found else '❌'}")
print(f"   • 설정 완료: {'✅' if 'config' in globals() else '❌'}")
print(f"   • 추출기 준비: {'✅' if 'text_extractor' in globals() else '❌'}")

print()
print("📁 디렉토리 구조:")
print(f"   • 입력: {config.input_docs_dir}")
print(f"   • 출력: {config.output_dir}")
print(f"   • 텍스트: {config.extracted_texts_dir}")
print(f"   • 결과: {config.test_results_dir}")

print()
print("🧪 사용 가능한 테스트 함수:")
print("   📄 단일 파일:")
print("      • test_single_file('파일경로')")
print("      • quick_test_pdf('pdf파일')")
print("      • quick_test_auto('파일경로')")
print()
print("   📊 배치 테스트:")
print("      • test_multiple_files()")
print("      • test_multiple_files(file_types=['pdf'])")
print("      • run_example_tests()")
print()
print("   📈 분석:")
print("      • analyze_extraction_performance()")
print("      • document_loader.scan_input_directory()")

print()
print("🎯 다음 단계:")
print("   1. 테스트할 문서를 input_docs 폴더에 추가")
print("   2. 디렉토리 스캔 셀 실행으로 파일 확인")
print("   3. 원하는 테스트 함수 실행")
print("   4. 결과 분석 및 성능 확인")

print()
print("💡 팁:")
print("   • 작은 파일부터 테스트하여 환경 검증")
print("   • PDF는 이미지/텍스트 기반에 따라 방법 선택")
print("   • 결과는 output 폴더에 자동 저장")
print("   • 성능 통계로 최적 방법 확인")

print()
print("🚀 테스트 준비 완료! 위의 함수들을 사용하여 문서 텍스트 추출을 테스트하세요!")

🎉 오픈소스 문서 텍스트 추출 테스트 환경 구성 완료!

🔧 환경 상태:
   • 라이브러리 로딩: ❌
   • Tesseract OCR: ✅
   • 설정 완료: ✅
   • 추출기 준비: ✅

📁 디렉토리 구조:
   • 입력: /home/admin/wkms-aws/jupyter_notebook/data/input_docs
   • 출력: /home/admin/wkms-aws/jupyter_notebook/data/output
   • 텍스트: /home/admin/wkms-aws/jupyter_notebook/data/output/extracted_texts
   • 결과: /home/admin/wkms-aws/jupyter_notebook/data/output/test_results

🧪 사용 가능한 테스트 함수:
   📄 단일 파일:
      • test_single_file('파일경로')
      • quick_test_pdf('pdf파일')
      • quick_test_auto('파일경로')

   📊 배치 테스트:
      • test_multiple_files()
      • test_multiple_files(file_types=['pdf'])
      • run_example_tests()

   📈 분석:
      • analyze_extraction_performance()
      • document_loader.scan_input_directory()

🎯 다음 단계:
   1. 테스트할 문서를 input_docs 폴더에 추가
   2. 디렉토리 스캔 셀 실행으로 파일 확인
   3. 원하는 테스트 함수 실행
   4. 결과 분석 및 성능 확인

💡 팁:
   • 작은 파일부터 테스트하여 환경 검증
   • PDF는 이미지/텍스트 기반에 따라 방법 선택
   • 결과는 output 폴더에 자동 저장
   • 성능 통계로 최적 방법 확인

🚀 테스트 준비 완료! 위의 함수들을 사용하여 문서 텍스트 추출을 테스트하세요!

In [12]:
# 🔤 OCR 추출 메소드 추가 (페이지별 로그 및 파일 저장)


def _extract_pdf_ocr(self, file_path):
    """OCR을 사용한 PDF 텍스트 추출 (페이지별 로그 및 파일 저장)"""
    import time
    from pathlib import Path
    # 지연 로딩: 무거운 라이브러리는 함수 내부에서 임포트하여 셀 시작 지연을 방지
    try:
        from pdf2image import convert_from_path  # type: ignore
    except Exception as e:
        raise RuntimeError("pdf2image 라이브러리가 필요합니다. 설치: pip install pdf2image") from e
    try:
        import pytesseract  # type: ignore
    except Exception as e:
        raise RuntimeError("pytesseract 라이브러리가 필요합니다. 설치: pip install pytesseract") from e
    from datetime import datetime


    # 설정 값 안전 획득
    ocr_cfg = getattr(self.config, 'ocr_config', {}) or {}
    ocr_enabled = ocr_cfg.get('enabled', True)
    ocr_lang = ocr_cfg.get('language', 'eng')
    ocr_dpi = ocr_cfg.get('dpi', 200)


    if not ocr_enabled:
        raise ValueError("OCR이 비활성화되어 있습니다.")


    file_path = Path(file_path)
    file_stem = file_path.stem


    try:
        print(f"🔄 PDF OCR 처리 시작: {file_path.name}", flush=True)
        print(f"   📊 DPI: {ocr_dpi}", flush=True)
        print(f"   🌍 언어: {ocr_lang}", flush=True)


        # PDF를 이미지로 변환
        print(f"   📄 PDF → 이미지 변환 중...", flush=True)
        start_conversion = time.time()
        images = convert_from_path(
        	file_path,
        	 dpi=ocr_dpi
        )
        conversion_time = time.time() - start_conversion


        print(f"   ✅ 변환 완료: {len(images)}페이지 ({conversion_time:.2f}초)", flush=True)


        metadata = {
            'pages': len(images),
            'extraction_method': 'ocr',
            'ocr_language': ocr_lang,
            'ocr_dpi': ocr_dpi,
            'conversion_time': conversion_time,
            'page_results': []
        }


        text_parts = []
        page_files = []  # 페이지별 파일 경로 저장


        # 각 페이지별 OCR 처리
        for page_num, image in enumerate(images, 1):
            page_start_time = time.time()


            try:
                print(f"   🔍 페이지 {page_num}/{len(images)} OCR 처리 중...", flush=True)


                # Tesseract OCR 실행
                page_text = pytesseract.image_to_string(
                    image,
                    lang=ocr_lang
                )


                page_processing_time = time.time() - page_start_time
                text_length = len(page_text.strip())


                # 페이지별 결과 저장
                page_result = {
                    'page_number': page_num,
                    'text_length': text_length,
                    'processing_time': page_processing_time,
                    'success': True,
                    'error': None
                }


                if page_text.strip():
                    # 페이지 텍스트 포맷팅
                    formatted_page_text = f"\n\n=== 페이지 {page_num} (OCR) ===\n{page_text}"
                    text_parts.append(formatted_page_text)


                    # 페이지별 파일 저장
                    page_file = self._save_page_text(file_stem, page_num, page_text)
                    if page_file:
                        page_files.append(page_file)
                        page_result['output_file'] = str(page_file)


                    print(f"      ✅ 완료: {text_length:,}자 추출 ({page_processing_time:.2f}초)", flush=True)
                    if hasattr(self, 'logger') and self.logger:
                        self.logger.info(f"OCR 페이지 {page_num} 성공: {text_length}자, {page_processing_time:.2f}초")
                else:
                    text_parts.append(f"\n\n=== 페이지 {page_num} (OCR - 텍스트 없음) ===\n")
                    print(f"      ⚠️ 텍스트 없음 ({page_processing_time:.2f}초)", flush=True)
                    if hasattr(self, 'logger') and self.logger:
                        self.logger.warning(f"OCR 페이지 {page_num}: 텍스트 없음")


                metadata['page_results'].append(page_result)


            except Exception as e:
                page_processing_time = time.time() - page_start_time
                error_msg = str(e)


                page_result = {
                    'page_number': page_num,
                    'text_length': 0,
                    'processing_time': page_processing_time,
                    'success': False,
                    'error': error_msg
                }
                metadata['page_results'].append(page_result)


                text_parts.append(f"\n\n=== 페이지 {page_num} (OCR 실패) ===\n오류: {error_msg}")
                print(f"      ❌ 실패: {error_msg} ({page_processing_time:.2f}초)", flush=True)
                if hasattr(self, 'logger') and self.logger:
                    self.logger.error(f"OCR 페이지 {page_num} 실패: {e}")


        # 전체 텍스트 결합
        full_text = '\n'.join(text_parts)


        # 전체 결과 파일 저장
        output_file = self._save_full_ocr_text(file_stem, full_text, metadata)


        # 통계 정보
        successful_pages = sum(1 for r in metadata['page_results'] if r['success'])
        total_chars = sum(r['text_length'] for r in metadata['page_results'])
        total_processing_time = sum(r['processing_time'] for r in metadata['page_results'])


        print(f"\n📊 OCR 처리 완료 - {file_path.name}", flush=True)
        print(f"   ✅ 성공 페이지: {successful_pages}/{len(images)}", flush=True)
        print(f"   📝 총 추출 텍스트: {total_chars:,}자", flush=True)
        print(f"   ⏱️ 총 처리시간: {total_processing_time:.2f}초", flush=True)
        print(f"   💾 저장된 파일:", flush=True)
        if output_file:
            print(f"      📄 전체 결과: {output_file.name}", flush=True)
        for i, page_file in enumerate(page_files, 1):
            print(f"      📑 페이지 {i}: {page_file.name}", flush=True)


        metadata.update({
            'successful_pages': successful_pages,
            'total_characters': total_chars,
            'total_processing_time': total_processing_time,
            'output_file': str(output_file) if output_file else None,
            'page_files': [str(f) for f in page_files]
        })


        return {
            'text': full_text,
            'metadata': metadata,
            'extraction_method': 'ocr'
        }


    except Exception as e:
        raise RuntimeError(f"OCR 처리 실패: {e}")


def _save_page_text(self, file_stem, page_num, page_text):
    """페이지별 텍스트를 파일로 저장"""
    from datetime import datetime
    try:
        # 파일명 생성: 원본파일명_page_001.txt
        page_filename = f"{file_stem}_page_{page_num:03d}.txt"
        # 올바른 저장 디렉토리 사용
        output_dir = getattr(self.config, 'extracted_texts_dir', None) or getattr(self.config, 'extracted_text_dir', None)
        if output_dir is None:
            raise RuntimeError("출력 디렉토리(extracted_texts_dir)가 설정되지 않았습니다.")
        page_file_path = output_dir / page_filename


        # 페이지 헤더 추가
        content = f"=== OCR 추출 텍스트 ===\n"
        content += f"원본 파일: {file_stem}\n"
        content += f"페이지 번호: {page_num}\n"
        content += f"추출 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
        content += f"텍스트 길이: {len(page_text.strip())}자\n"
        ocr_cfg = getattr(self.config, 'ocr_config', {}) or {}
        content += f"언어: {ocr_cfg.get('language', 'eng')}\n"
        content += f"DPI: {ocr_cfg.get('dpi', 200)}\n"
        content += "=" * 50 + "\n\n"
        content += page_text.strip()


        # 파일 저장
        with open(page_file_path, 'w', encoding='utf-8') as f:
            f.write(content)


        return page_file_path


    except Exception as e:
        if hasattr(self, 'logger') and self.logger:
            self.logger.error(f"페이지 {page_num} 텍스트 저장 실패: {e}")
        return None


def _save_full_ocr_text(self, file_stem, full_text, metadata):
    """전체 OCR 결과를 파일로 저장"""
    from datetime import datetime
    import json
    try:
        # 파일명 생성: 원본파일명_ocr_full.txt
        full_filename = f"{file_stem}_ocr_full.txt"
        output_dir = getattr(self.config, 'extracted_texts_dir', None) or getattr(self.config, 'extracted_text_dir', None)
        if output_dir is None:
            raise RuntimeError("출력 디렉토리(extracted_texts_dir)가 설정되지 않았습니다.")
        full_file_path = output_dir / full_filename


        # 헤더 정보 생성
        content = f"=== OCR 전체 추출 결과 ===\n"
        content += f"원본 파일: {file_stem}\n"
        content += f"추출 방법: OCR (Tesseract)\n"
        content += f"처리 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
        content += f"총 페이지: {metadata['pages']}\n"
        content += f"성공 페이지: {metadata.get('successful_pages', 0)}\n"
        content += f"총 텍스트: {metadata.get('total_characters', 0):,}자\n"
        content += f"처리 시간: {metadata.get('total_processing_time', 0):.2f}초\n"
        content += f"언어 설정: {metadata.get('ocr_language', 'eng')}\n"
        content += f"DPI 설정: {metadata.get('ocr_dpi', 200)}\n"
        content += "=" * 80 + "\n\n"


        # 페이지별 처리 결과 요약
        content += "📊 페이지별 처리 결과:\n"
        for page_result in metadata.get('page_results', []):
            status = "✅" if page_result['success'] else "❌"
            content += f"   페이지 {page_result['page_number']}: {status} "
            content += f"{page_result['text_length']:,}자 "
            content += f"({page_result['processing_time']:.2f}초)\n"
            if not page_result['success']:
                content += f"      오류: {page_result['error']}\n"
        content += "\n" + "=" * 80 + "\n\n"


        # 전체 텍스트 추가
        content += full_text


        # 파일 저장
        with open(full_file_path, 'w', encoding='utf-8') as f:
            f.write(content)


        # 추가로 JSON 메타데이터 파일도 저장
        metadata_filename = f"{file_stem}_ocr_metadata.json"
        metadata_dir = getattr(self.config, 'metadata_dir', None) or getattr(self.config, 'output_dir', None)
        if metadata_dir is None:
            metadata_dir = output_dir
        metadata_file_path = metadata_dir / metadata_filename


        with open(metadata_file_path, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, ensure_ascii=False, indent=2, default=str)


        return full_file_path


    except Exception as e:
        if hasattr(self, 'logger') and self.logger:
            self.logger.error(f"OCR 전체 결과 저장 실패: {e}")
        return None


print("🔍 OCR 추출 작업을 시작합니다...", flush=True)
# 등록할 추출기 클래스 확인 (TextExtractor 우선, 없으면 DocumentTextExtractor 사용)
try:
    ExtractorClass = TextExtractor  # 기존 이름 호환
except NameError:
    try:
        ExtractorClass = DocumentTextExtractor  # 새 이름
    except NameError:
        ExtractorClass = None


if ExtractorClass is None:
    raise NameError("OCR 메소드를 등록할 대상 클래스(TextExtractor 또는 DocumentTextExtractor)를 찾을 수 없습니다. 먼저 추출기 클래스를 정의하세요.")


# 이미 등록되었는지 확인하여 중복 등록 방지
if getattr(ExtractorClass, "_ocr_methods_registered", False):
    print("ℹ️ 이미 OCR 추출 메소드가 등록되어 있습니다.", flush=True)
else:
    # OCR 관련 메소드 등록
    ExtractorClass._extract_pdf_ocr = _extract_pdf_ocr
    ExtractorClass._save_page_text = _save_page_text
    ExtractorClass._save_full_ocr_text = _save_full_ocr_text
    setattr(ExtractorClass, "_ocr_methods_registered", True)
    print("🔍 OCR 추출 메소드 추가 완료!", flush=True)
    print("   📊 페이지별 실시간 로그", flush=True)
    print("   💾 개별 페이지 파일 저장", flush=True)
    print("   📄 전체 결과 통합 파일", flush=True)

🔍 OCR 추출 작업을 시작합니다...
🔍 OCR 추출 메소드 추가 완료!
   📊 페이지별 실시간 로그
   💾 개별 페이지 파일 저장
   📄 전체 결과 통합 파일
🔍 OCR 추출 메소드 추가 완료!
   📊 페이지별 실시간 로그
   💾 개별 페이지 파일 저장
   📄 전체 결과 통합 파일


In [13]:
# ✅ 테스트 PDF에 대해 실제 OCR을 실행하고, 페이지별 로그와 저장 결과를 출력하는 러너 셀


from pathlib import Path


print("🚀 테스트 PDF OCR 실행 러너 시작", flush=True)


# 입력 및 출력 경로 설정
test_pdf_path = Path("/home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf")


if 'config' in globals() and hasattr(config, 'extracted_texts_dir'):

    output_dir = config.extracted_texts_dir

elif 'config' in globals() and hasattr(config, 'extracted_text_dir'):

    output_dir = config.extracted_text_dir

else:

    output_dir = Path("/home/admin/wkms-aws/jupyter_notebook/data/opensource_output/extracted_texts")


output_dir.mkdir(parents=True, exist_ok=True)


print(f"📄 입력 파일: {test_pdf_path}", flush=True)
print(f"💾 출력 디렉토리: {output_dir}", flush=True)


# 존재 여부 확인
if not test_pdf_path.exists():

    raise FileNotFoundError(f"입력 파일이 존재하지 않습니다: {test_pdf_path}")


# 추출기 인스턴스 확인/생성
extractor = None


if 'text_extractor' in globals():

    extractor = text_extractor

else:

    # 클래스 이름 호환 처리
    try:

        extractor = DocumentTextExtractor(config)

    except NameError:

        try:

            extractor = TextExtractor(config)

        except NameError as e:

            raise NameError("추출기 클래스(DocumentTextExtractor 또는 TextExtractor)가 정의되지 않았습니다. 먼저 해당 셀을 실행하세요.") from e


# OCR 설정이 꺼져 있으면 이번 실행에 한해 임시로 켭니다.
prev_ocr_enabled = None
ocr_cfg = getattr(extractor.config, 'ocr_config', {}) if hasattr(extractor, 'config') else {}
prev_ocr_enabled = ocr_cfg.get('enabled', True)
if not prev_ocr_enabled:

    print("ℹ️ 현재 OCR이 비활성화되어 있어, 이번 실행 동안만 임시로 활성화합니다.", flush=True)

    extractor.config.ocr_config['enabled'] = True


try:

    print("🔍 OCR 추출 작업을 시작합니다 (페이지별 상세 로그는 아래에 표시됩니다)…", flush=True)

    result = extractor._extract_pdf_ocr(test_pdf_path)

    print("\n✅ OCR 추출 완료", flush=True)

except Exception as e:

    print(f"❌ OCR 실행 중 오류: {e}", flush=True)

    raise

finally:

    # 설정 복구
    if prev_ocr_enabled is not None:

        extractor.config.ocr_config['enabled'] = prev_ocr_enabled


# 저장 파일 요약 출력
try:

    meta = result.get('metadata', {}) if isinstance(result, dict) else {}

    output_file = meta.get('output_file')

    page_files = meta.get('page_files', [])

    text_len = meta.get('total_characters', len(result.get('text', '') if isinstance(result, dict) else ''))

    print("\n📦 저장 결과 요약", flush=True)

    if output_file:

        print(f"   📄 전체 결과 파일: {Path(output_file).name}", flush=True)

    if page_files:

        print(f"   📑 페이지별 파일 수: {len(page_files)}", flush=True)

        sample_list = [Path(p).name for p in page_files[:5]]

        more = ' ...' if len(page_files) > 5 else ''

        print(f"      예시: {', '.join(sample_list)}{more}", flush=True)

    print(f"   📝 총 텍스트 길이: {text_len:,}자", flush=True)

    print(f"   📂 출력 디렉토리: {output_dir}", flush=True)

except Exception as e:

    print(f"⚠️ 저장 결과 요약 출력 중 오류: {e}", flush=True)

    

# 미리보기 (과도한 출력 방지)
try:

    text = result.get('text', '') if isinstance(result, dict) else ''

    if text:

        preview = text.strip().splitlines()

        head = '\n'.join(preview[:20])  # 처음 20줄 미리보기

        print("\n🔎 텍스트 미리보기 (처음 20줄)", flush=True)

        print(head[:2000], flush=True)  # 최대 2000자 제한

    else:

        print("(미리보기 텍스트가 없습니다)", flush=True)

except Exception as e:

    print(f"⚠️ 미리보기 출력 중 오류: {e}", flush=True)

🚀 테스트 PDF OCR 실행 러너 시작
📄 입력 파일: /home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf
📄 입력 파일: /home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf
💾 출력 디렉토리: /home/admin/wkms-aws/jupyter_notebook/data/output/extracted_texts
🔍 OCR 추출 작업을 시작합니다 (페이지별 상세 로그는 아래에 표시됩니다)…
🔄 PDF OCR 처리 시작: test.pdf
   📊 DPI: 300
   🌍 언어: kor+eng
   📄 PDF → 이미지 변환 중...
💾 출력 디렉토리: /home/admin/wkms-aws/jupyter_notebook/data/output/extracted_texts
🔍 OCR 추출 작업을 시작합니다 (페이지별 상세 로그는 아래에 표시됩니다)…
🔄 PDF OCR 처리 시작: test.pdf
   📊 DPI: 300
   🌍 언어: kor+eng
   📄 PDF → 이미지 변환 중...
   ✅ 변환 완료: 27페이지 (5.55초)
   🔍 페이지 1/27 OCR 처리 중...
   ✅ 변환 완료: 27페이지 (5.55초)
   🔍 페이지 1/27 OCR 처리 중...
      ✅ 완료: 2,973자 추출 (18.28초)
   🔍 페이지 2/27 OCR 처리 중...
      ✅ 완료: 2,973자 추출 (18.28초)
   🔍 페이지 2/27 OCR 처리 중...
      ✅ 완료: 3,603자 추출 (16.98초)
   🔍 페이지 3/27 OCR 처리 중...
      ✅ 완료: 3,603자 추출 (16.98초)
   🔍 페이지 3/27 OCR 처리 중...
      ✅ 완료: 3,488자 추출 (15.52초)
   🔍 페이지 4/27 OCR 처리 중...
      ✅ 완료: 3,488자 추출 (15.52초)
   🔍 페이지 4/

In [15]:
# 🔤 나머지 문서 형식 추출 메소드 추가


def _extract_from_docx(self, file_path):
    """Word 문서에서 텍스트 추출"""
    # 지연 임포트
    try:
        from docx import Document  # type: ignore
    except Exception as e:
        raise RuntimeError("python-docx 라이브러리가 필요합니다. 설치: pip install python-docx") from e
    doc = Document(file_path)


    # 메타데이터 추출
    core_props = doc.core_properties
    metadata = {
        'extraction_method': 'python-docx',
        'title': core_props.title or '',
        'author': core_props.author or '',
        'created': str(core_props.created) if core_props.created else '',
        'modified': str(core_props.modified) if core_props.modified else '',
        'paragraphs': len(doc.paragraphs),
        'tables': len(doc.tables)
    }


    # 텍스트 추출
    text_parts = []


    # 단락 텍스트
    for para in doc.paragraphs:
        if para.text.strip():
            text_parts.append(para.text)


    # 테이블 텍스트
    for table_num, table in enumerate(doc.tables, 1):
        table_text = f"\n\n=== 테이블 {table_num} ===\n"
        for row in table.rows:
            row_cells = [cell.text.strip() for cell in row.cells]
            table_text += " | ".join(row_cells) + "\n"
        text_parts.append(table_text)


    full_text = '\n'.join(text_parts)


    return {
        'text': full_text,
        'metadata': metadata,
        'extraction_method': 'python-docx'
    }


def _extract_from_pptx(self, file_path):
    """PowerPoint에서 텍스트 추출"""
    # 지연 임포트
    try:
        from pptx import Presentation  # type: ignore
    except Exception as e:
        raise RuntimeError("python-pptx 라이브러리가 필요합니다. 설치: pip install python-pptx") from e
    prs = Presentation(file_path)


    metadata = {
        'extraction_method': 'python-pptx',
        'slides': len(prs.slides)
    }


    text_parts = []


    for slide_num, slide in enumerate(prs.slides, 1):
        slide_text = f"\n\n=== 슬라이드 {slide_num} ===\n"


        for shape in slide.shapes:
            if hasattr(shape, "text") and shape.text.strip():
                slide_text += shape.text + "\n"


        text_parts.append(slide_text)


    full_text = '\n'.join(text_parts)


    return {
        'text': full_text,
        'metadata': metadata,
        'extraction_method': 'python-pptx'
    }


def _extract_from_excel(self, file_path):
    """Excel에서 텍스트 추출"""
    try:
        import pandas as pd  # type: ignore
    except Exception as e:
        raise RuntimeError("pandas 라이브러리가 필요합니다. 설치: pip install pandas openpyxl") from e
    try:
        # 모든 시트 읽기
        excel_data = pd.read_excel(file_path, sheet_name=None)


        metadata = {
            'extraction_method': 'openpyxl+pandas',
            'sheets': list(excel_data.keys()),
            'sheet_count': len(excel_data)
        }


        text_parts = []


        for sheet_name, df in excel_data.items():
            sheet_text = f"\n\n=== 시트: {sheet_name} ===\n"
            sheet_text += df.to_string(index=False)
            text_parts.append(sheet_text)


        full_text = '\n'.join(text_parts)


        return {
            'text': full_text,
            'metadata': metadata,
            'extraction_method': 'openpyxl+pandas'
        }


    except Exception as e:
        raise RuntimeError(f"Excel 파일 읽기 실패: {e}")


def _extract_from_text(self, file_path):
    """텍스트 파일에서 내용 추출"""
    # DocumentLoader의 _read_text_file_fast 활용
    try:
        DocumentLoader  # type: ignore
    except NameError:
        from pathlib import Path  # fallback to avoid NameError at def time
    doc_loader = DocumentLoader(self.config)  # type: ignore
    text_content = doc_loader._read_text_file_fast(file_path)  # type: ignore


    metadata = {
        'extraction_method': 'text_file',
        'lines': len(text_content.split('\n'))
    }


    return {
        'text': text_content,
        'metadata': metadata,
        'extraction_method': 'text_file'
    }


def _extract_from_html(self, file_path):
    """HTML에서 텍스트 추출"""
    # HTML 파일 읽기
    try:
        DocumentLoader  # type: ignore
    except NameError:
        pass
    doc_loader = DocumentLoader(self.config)  # type: ignore
    html_content = doc_loader._read_text_file_fast(file_path)  # type: ignore


    # BeautifulSoup으로 정제된 텍스트 추출
    processed_text = self._process_html_content(html_content)


    metadata = {
        'extraction_method': 'beautifulsoup+html2text',
        'content_length': len(html_content)
    }


    return {
        'text': processed_text,
        'metadata': metadata,
        'extraction_method': 'beautifulsoup+html2text'
    }


def _extract_from_image(self, file_path):
    """이미지에서 OCR로 텍스트 추출"""
    if not self.config.processing_config['ocr_enabled']:
        raise ValueError("OCR이 비활성화되어 있습니다.")
    
    try:
        from PIL import Image  # type: ignore
    except Exception as e:
        raise RuntimeError("Pillow 라이브러리가 필요합니다. 설치: pip install Pillow") from e
    try:
        import pytesseract  # type: ignore
    except Exception as e:
        raise RuntimeError("pytesseract 라이브러리가 필요합니다. 설치: pip install pytesseract") from e


    try:
        image = Image.open(file_path)


        # OCR 실행
        text = pytesseract.image_to_string(
            image,
            lang=self.config.processing_config['ocr_language']
        )


        metadata = {
            'extraction_method': 'tesseract_ocr',
            'image_size': image.size,
            'image_mode': image.mode,
            'ocr_language': self.config.processing_config['ocr_language']
        }


        return {
            'text': text,
            'metadata': metadata,
            'extraction_method': 'tesseract_ocr'
        }


    except Exception as e:
        raise RuntimeError(f"이미지 OCR 실패: {e}")


def _process_html_content(self, html_content):
    """HTML 콘텐츠 정제"""
    # 지연 임포트
    try:
        from bs4 import BeautifulSoup  # type: ignore
    except Exception as e:
        raise RuntimeError("beautifulsoup4 라이브러리가 필요합니다. 설치: pip install beautifulsoup4 html2text") from e
    try:
        import re  # stdlib
    except Exception:
        pass


    try:
        # BeautifulSoup으로 파싱
        soup = BeautifulSoup(html_content, 'html.parser')


        # 테이블 처리
        tables = soup.find_all('table')
        for table in tables:
            table_text = self._convert_html_table_to_text(table)
            table.replace_with(table_text)


        # 불필요한 태그 제거
        for tag in soup(['script', 'style', 'meta', 'link', 'head']):
            tag.decompose()


        # 텍스트 추출
        text = soup.get_text()


        # 정규표현식으로 정제
        import re as _re
        text = _re.sub(r'<!--.*?-->', '', text, flags=_re.DOTALL)  # HTML 주석
        text = _re.sub(r'\n{3,}', '\n\n', text)  # 연속 개행
        text = _re.sub(r'[ \t]+', ' ', text)  # 연속 공백


        return text.strip()


    except Exception as e:
        self.logger.warning(f"HTML 정제 중 오류: {e}")
        return html_content


def _convert_html_table_to_text(self, table):
    """HTML 테이블을 텍스트로 변환"""
    try:
        rows = table.find_all('tr')
        if not rows:
            return "\n[빈 테이블]\n"


        table_text = "\n\n=== 테이블 ===\n"
        for row in rows:
            cells = row.find_all(['td', 'th'])
            row_text = []
            for cell in cells:
                cell_text = cell.get_text().strip()
                row_text.append(cell_text)
            if row_text:
                table_text += " | ".join(row_text) + "\n"


        return table_text


    except Exception:
        return "\n[테이블 처리 오류]\n"


def _format_tables_as_text(self, tables, page_num):
    """pdfplumber 테이블을 텍스트로 포맷"""
    if not tables:
        return ""


    table_text = f"\n\n=== 페이지 {page_num} 테이블 ===\n"


    for table_num, table in enumerate(tables, 1):
        table_text += f"\n--- 테이블 {table_num} ---\n"
        for row in table:
            if row:
                clean_row = [str(cell).strip() if cell else "" for cell in row]
                table_text += " | ".join(clean_row) + "\n"


    return table_text


def extract_text_from_document(self, document_info):
    """배치 프로세서와의 호환성을 위한 래퍼 메소드"""
    from pathlib import Path
    try:
        file_path = document_info.get('file_path')
        if not file_path:
            raise ValueError("문서 경로가 없습니다.")


        # 문서 타입에 따른 추출 방법 결정
        file_type = document_info.get('file_type', 'unknown')


        # 이미지 기반 PDF 감지 로직
        method = 'auto'
        if file_type == 'pdf':
            # 기존 파일 내용이 비어있거나 매우 적으면 이미지 기반일 가능성
            existing_content = document_info.get('file_content', '')
            if not existing_content or len(existing_content.strip()) < 100:
                print(f"⚠️ 이미지 기반 PDF로 판단됨: {Path(file_path).name}", flush=True)
                method = 'ocr'  # OCR 강제 사용


        # 텍스트 추출 실행
        result = self.extract_text(file_path, method=method)


        # 배치 프로세서 호환 형식으로 결과 포맷팅
        return {
            'success': result.get('success', False),
            'extracted_text': result.get('text', ''),
            'extraction_method': result.get('extraction_method', method),
            'metadata': result.get('metadata', {}),
            'processing_time': result.get('processing_time', 0),
            'file_path': file_path,
            'file_type': file_type,
            'error': result.get('error')
        }


    except Exception as e:
        return {
            'success': False,
            'extracted_text': '',
            'extraction_method': 'failed',
            'metadata': {},
            'processing_time': 0,
            'file_path': document_info.get('file_path', ''),
            'file_type': document_info.get('file_type', 'unknown'),
            'error': str(e)
        }


# 메소드 등록 대상 클래스 확인 (TextExtractor 우선, 없으면 DocumentTextExtractor 사용)
try:
    ExtractorClass = TextExtractor  # 기존 이름
except NameError:
    try:
        ExtractorClass = DocumentTextExtractor  # 새 이름
    except NameError:
        ExtractorClass = None


if ExtractorClass is None:
    raise NameError("메소드를 등록할 대상 클래스(TextExtractor 또는 DocumentTextExtractor)를 찾을 수 없습니다. 먼저 추출기 클래스를 정의하세요.")


# 이미 등록되었는지 확인하여 중복 등록 방지
if getattr(ExtractorClass, "_other_methods_registered", False):
    print("ℹ️ 기타 문서 형식 추출 메소드는 이미 등록되어 있습니다.", flush=True)
else:
    # 메소드 추가
    ExtractorClass._extract_from_docx = _extract_from_docx
    ExtractorClass._extract_from_pptx = _extract_from_pptx
    ExtractorClass._extract_from_excel = _extract_from_excel
    ExtractorClass._extract_from_text = _extract_from_text
    ExtractorClass._extract_from_html = _extract_from_html
    ExtractorClass._extract_from_image = _extract_from_image
    ExtractorClass._process_html_content = _process_html_content
    ExtractorClass._convert_html_table_to_text = _convert_html_table_to_text
    ExtractorClass._format_tables_as_text = _format_tables_as_text
    ExtractorClass.extract_text_from_document = extract_text_from_document
    setattr(ExtractorClass, "_other_methods_registered", True)
    print("✅ 모든 텍스트 추출 메소드 추가 완료!", flush=True)
    print("=" * 70, flush=True)
    print("📋 지원하는 추출 방법:", flush=True)
    print("   📄 PDF: PyPDF2, pdfplumber, OCR (페이지별 저장)", flush=True)
    print("   📝 Word: python-docx", flush=True)
    print("   🎨 PowerPoint: python-pptx", flush=True)
    print("   📊 Excel: pandas + openpyxl", flush=True)
    print("   🌐 HTML: BeautifulSoup + html2text", flush=True)
    print("   🖼️ 이미지: Tesseract OCR", flush=True)
    print("   📄 텍스트: 인코딩 자동 감지", flush=True)
    print(f"\n🆕 OCR 개선사항:", flush=True)
    print(f"   📊 페이지별 실시간 진행 로그", flush=True)
    print(f"   💾 페이지별 개별 텍스트 파일 저장", flush=True)
    print(f"   📄 전체 결과 통합 파일 저장", flush=True)
    try:
        print(f"   💾 저장 경로: {config.extracted_text_dir}", flush=True)
    except Exception:
        pass
    print(f"\n💡 사용법:", flush=True)
    print(f"   - text_extractor.extract_text(file_path): 자동 텍스트 추출", flush=True)
    print(f"   - text_extractor.extract_text(file_path, method='ocr'): OCR 강제 사용", flush=True)
    print(f"   - text_extractor.extract_text_from_document(doc_info): 배치 처리용", flush=True)

✅ 모든 텍스트 추출 메소드 추가 완료!
📋 지원하는 추출 방법:
   📄 PDF: PyPDF2, pdfplumber, OCR (페이지별 저장)
   📝 Word: python-docx
   🎨 PowerPoint: python-pptx
   📊 Excel: pandas + openpyxl
   🌐 HTML: BeautifulSoup + html2text
📋 지원하는 추출 방법:
   📄 PDF: PyPDF2, pdfplumber, OCR (페이지별 저장)
   📝 Word: python-docx
   🎨 PowerPoint: python-pptx
   📊 Excel: pandas + openpyxl
   🌐 HTML: BeautifulSoup + html2text


   🖼️ 이미지: Tesseract OCR
   📄 텍스트: 인코딩 자동 감지

🆕 OCR 개선사항:
   📊 페이지별 실시간 진행 로그
   💾 페이지별 개별 텍스트 파일 저장
   📄 전체 결과 통합 파일 저장

💡 사용법:
   - text_extractor.extract_text(file_path): 자동 텍스트 추출
   - text_extractor.extract_text(file_path, method='ocr'): OCR 강제 사용
   - text_extractor.extract_text_from_document(doc_info): 배치 처리용
   📄 텍스트: 인코딩 자동 감지

🆕 OCR 개선사항:
   📊 페이지별 실시간 진행 로그
   💾 페이지별 개별 텍스트 파일 저장
   📄 전체 결과 통합 파일 저장

💡 사용법:
   - text_extractor.extract_text(file_path): 자동 텍스트 추출
   - text_extractor.extract_text(file_path, method='ocr'): OCR 강제 사용
   - text_extractor.extract_text_from_document(doc_info): 배치 처리용


In [16]:
# 🚀 OCR 실행 러너: test.pdf를 대상으로 페이지별 로그와 결과 저장 수행


from pathlib import Path
import time


print("🟢 OCR 테스트 러너 시작", flush=True)


# 입력 파일과 출력 디렉토리 설정
test_pdf_path = Path("/home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf")


# 구성에서 출력 디렉토리 확인 (없으면 기본 경로로 대체)
if 'config' in globals() and hasattr(config, 'extracted_text_dir') and config.extracted_text_dir:
    output_dir = config.extracted_text_dir
else:
    output_dir = Path("/home/admin/wkms-aws/jupyter_notebook/data/opensource_output/extracted_texts")


output_dir.mkdir(parents=True, exist_ok=True)


print(f"📄 입력 파일: {test_pdf_path}", flush=True)
print(f"📂 출력 경로: {output_dir}", flush=True)


# 입력 파일 유효성 검사
if not test_pdf_path.exists():
    raise FileNotFoundError(f"입력 파일이 존재하지 않습니다: {test_pdf_path}")


# 추출기 인스턴스 준비 (text_extractor ▶ extractor ▶ 새로 생성 순)
extractor = None
if 'text_extractor' in globals() and text_extractor:
    extractor = text_extractor
elif 'extractor' in globals() and extractor:  # noqa: F823 (VSCode placeholder)
    extractor = extractor
else:
    try:
        extractor = DocumentTextExtractor(config)  # type: ignore
    except NameError:
        # 클래스 이름이 다를 경우 대비
        extractor = TextExtractor(config)  # type: ignore


# OCR 설정 보장 (기존 값 보존 후 복원)
ocr_cfg = getattr(config, 'processing_config', {})
prev_ocr_enabled = ocr_cfg.get('ocr_enabled', True)
ocr_cfg['ocr_enabled'] = True


print("🔎 실행 정보:", flush=True)
print(f"   • OCR 활성화: {ocr_cfg.get('ocr_enabled')}", flush=True)
print(f"   • 언어: {ocr_cfg.get('ocr_language', 'eng')}", flush=True)
print(f"   • DPI: {ocr_cfg.get('ocr_dpi', 200)}", flush=True)


start_ts = time.time()
page_files = []
output_file = None
meta = {}
text = ""


try:
    # 메서드 등록 여부와 상관없이 공개 API로 OCR 강제 실행
    print("🔄 OCR 실행 중 (페이지별 진행 로그는 아래에 표시됩니다)…", flush=True)
    result = extractor.extract_text(str(test_pdf_path), method='ocr')
    
    # 결과 정리
    success = result.get('success', True)
    text = result.get('text', '')
    meta = result.get('metadata', {})
    output_file = Path(meta.get('output_file')) if meta.get('output_file') else None
    page_files = [Path(p) for p in meta.get('page_files', []) if p]
    
    print("\n✅ OCR 실행 완료", flush=True)
    print(f"   • 성공 여부: {success}", flush=True)
    print(f"   • 총 텍스트 길이: {len(text):,}자", flush=True)
    print(f"   • 저장된 페이지 파일 수: {len(page_files)}", flush=True)
    if output_file:
        print(f"   • 전체 결과 파일: {output_file}", flush=True)
    else:
        # 관례상 전체 결과 파일명 추정
        candidate = output_dir / f"{test_pdf_path.stem}_ocr_full.txt"
        if candidate.exists():
            output_file = candidate
            print(f"   • 전체 결과 파일(추정): {output_file}", flush=True)
except Exception as e:
    print(f"❌ OCR 실행 중 오류: {e}", flush=True)
finally:
    # 설정 복원
    ocr_cfg['ocr_enabled'] = prev_ocr_enabled


elapsed = time.time() - start_ts
print(f"⏱️ 총 소요 시간: {elapsed:.2f}초", flush=True)


# 산출물 요약
try:
    # 출력 디렉토리에 생성된 파일들 나열
    related_files = sorted(output_dir.glob(f"{test_pdf_path.stem}*"))
    print(f"\n💾 생성된 파일 목록 ({len(related_files)}개):", flush=True)
    for p in related_files[:15]:
        print(f"   • {p.name}", flush=True)
    if len(related_files) > 15:
        print(f"   … (총 {len(related_files)}개)", flush=True)


    # 전체 결과 파일 미리보기
    if output_file and output_file.exists():
        print("\n🔎 전체 결과 파일 미리보기 (상위 500자):", flush=True)
        head = ""
        try:
            with open(output_file, 'r', encoding='utf-8') as f:
                head = f.read(500)
        except Exception:
            with open(output_file, 'r', encoding='cp949', errors='ignore') as f:
                head = f.read(500)
        print(head if head else "(미리보기 없음)", flush=True)
    else:
        print("(전체 결과 파일을 찾지 못했습니다)", flush=True)
except Exception as e:
    print(f"⚠️ 산출물 요약 중 오류: {e}", flush=True)


print("🏁 OCR 테스트 러너 종료", flush=True)

🟢 OCR 테스트 러너 시작
📄 입력 파일: /home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf
📂 출력 경로: /home/admin/wkms-aws/jupyter_notebook/data/opensource_output/extracted_texts
🔎 실행 정보:
   • OCR 활성화: True
   • 언어: eng
   • DPI: 200
🔄 OCR 실행 중 (페이지별 진행 로그는 아래에 표시됩니다)…
🔄 텍스트 추출 시작: test.pdf
   📄 파일 타입: pdf
   🔧 추출 방법: ocr
🔄 PDF OCR 처리 시작: test.pdf
   📊 DPI: 300
   🌍 언어: kor+eng
   📄 PDF → 이미지 변환 중...
   ✅ 변환 완료: 27페이지 (5.21초)
   🔍 페이지 1/27 OCR 처리 중...
      ✅ 완료: 2,973자 추출 (18.97초)
   🔍 페이지 2/27 OCR 처리 중...
      ✅ 완료: 3,603자 추출 (17.44초)
   🔍 페이지 3/27 OCR 처리 중...
      ✅ 완료: 3,488자 추출 (15.85초)
   🔍 페이지 4/27 OCR 처리 중...
      ✅ 완료: 3,517자 추출 (17.46초)
   🔍 페이지 5/27 OCR 처리 중...
      ✅ 완료: 2,414자 추출 (11.06초)
   🔍 페이지 6/27 OCR 처리 중...
      ✅ 완료: 3,451자 추출 (18.56초)
   🔍 페이지 7/27 OCR 처리 중...
      ✅ 완료: 3,482자 추출 (16.21초)
   🔍 페이지 8/27 OCR 처리 중...
      ✅ 완료: 3,414자 추출 (16.94초)
   🔍 페이지 9/27 OCR 처리 중...
      ✅ 완료: 3,478자 추출 (16.78초)
   🔍 페이지 10/27 OCR 처리 중...
      ✅ 완료: 3,172자 추출 (17.27초)
   🔍

## 🧹 5단계: 텍스트 전처리 및 청킹 (백엔드 호환)

In [ ]:
# 🧹 문서 전처리 및 정제 (백엔드 tools.py 기반)


class DocumentPreprocessor:
    """문서 텍스트 전처리 및 정제 클래스"""


    def __init__(self, config):
        self.config = config
        self.logger = config.logger
        self.tokenizer = config.tokenizer


    def preprocess_text(self, text, apply_all=True):
        """
        텍스트 전처리를 수행합니다.

        Args:
            text: 원본 텍스트
            apply_all: 모든 전처리 단계를 적용할지 여부
        """
        if not text or not text.strip():
            return ""


        processed_text = text
        processing_steps = []


        try:
            # 1. 유니코드 정규화
            if self.config.processing_config['normalize_unicode']:
                import unicodedata
                processed_text = unicodedata.normalize('NFKC', processed_text)
                processing_steps.append("unicode_normalization")


            # 2. HTML/XML 태그 제거 및 정제 (백엔드 process_file 기반)
            if '<' in processed_text and '>' in processed_text:
                processed_text = self._clean_html_content(processed_text)
                processing_steps.append("html_cleaning")


            # 3. 공백 및 개행 정리
            if self.config.processing_config['clean_whitespace']:
                processed_text = self._clean_whitespace(processed_text)
                processing_steps.append("whitespace_cleaning")


            # 4. 빈 줄 제거
            if self.config.processing_config['remove_empty_lines']:
                processed_text = self._remove_empty_lines(processed_text)
                processing_steps.append("empty_lines_removal")


            # 5. 특수 문자 및 기호 정리
            processed_text = self._clean_special_characters(processed_text)
            processing_steps.append("special_characters_cleaning")


            # 6. 중복 공백 제거
            import re
            processed_text = re.sub(r'\s+', ' ', processed_text)
            processed_text = processed_text.strip()


            return {
                'original_text': text,
                'processed_text': processed_text,
                'original_length': len(text),
                'processed_length': len(processed_text),
                'processing_steps': processing_steps,
                'compression_ratio': len(processed_text) / len(text) if len(text) > 0 else 0
            }


        except Exception as e:
            self.logger.error(f"텍스트 전처리 실패: {e}")
            return {
                'original_text': text,
                'processed_text': text,
                'original_length': len(text),
                'processed_length': len(text),
                'processing_steps': [],
                'compression_ratio': 1.0,
                'error': str(e)
            }


    def _clean_html_content(self, content):
        """HTML 콘텐츠 정제 (백엔드 tools.py의 process_file 함수와 동일)"""
        try:
            # BeautifulSoup으로 HTML 파싱
            try:
                from bs4 import BeautifulSoup  # type: ignore
            except Exception as e:
                self.logger.warning("beautifulsoup4가 필요합니다. pip install beautifulsoup4 html2text")
                return content
            import re
            soup = BeautifulSoup(content, 'html.parser')


            # 테이블 처리
            tables = soup.find_all('table')
            for table in tables:
                table_text = self._process_table_to_text(table)
                table.replace_with(table_text)


            # 불필요한 태그 제거
            for tag in soup(['script', 'style', 'meta', 'link']):
                tag.decompose()


            # 텍스트 추출 및 정제
            text = soup.get_text()


            # 정규표현식으로 정제 (백엔드와 동일)
            text = re.sub(r'!\[\]\(figures/\d+\)', '', text)  # 그림 참조 제거
            text = re.sub(r'<!--\s*FigureContent="([^"]+)"\s*-->', r'\1', text)  # 그림 내용 추출
            text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)  # HTML 주석 제거
            text = re.sub(r'<[^>]+>\s*(.*?)\s*</[^>]+>', r'\1', text)  # 나머지 태그 제거
            text = text.replace('<figure>', '').replace('</figure>', '')
            text = text.replace('<table>', '').replace('</table>', '')
            text = re.sub(r'\n{3,}', '\n\n', text)  # 연속 개행 정리


            return text


        except Exception as e:
            self.logger.warning(f"HTML 정제 중 오류: {e}")
            return content


    def _process_table_to_text(self, table_html):
        """HTML 테이블을 텍스트로 변환 (백엔드 tools.py와 동일 로직)"""
        try:
            from bs4 import BeautifulSoup  # type: ignore
            soup = BeautifulSoup(str(table_html), 'html.parser')
            table = soup.find('table')


            if not table:
                return ""


            rows = table.find_all('tr')
            if not rows:
                return ""


            # 테이블 데이터 추출
            table_data = []
            for row in rows:
                cells = row.find_all(['td', 'th'])
                row_data = []
                for cell in cells:
                    cell_text = cell.get_text().strip()
                    row_data.append(cell_text)
                if row_data:
                    table_data.append(row_data)


            if not table_data:
                return ""


            # 컬럼 너비 계산
            max_cols = max(len(row) for row in table_data)
            col_widths = []


            for col_idx in range(max_cols):
                max_width = 0
                for row in table_data:
                    if col_idx < len(row):
                        max_width = max(max_width, len(row[col_idx]))
                col_widths.append(min(max_width, 30))  # 최대 30자로 제한


            # 텍스트 테이블 생성
            output_text = "\n"


            # 상단 경계
            output_text += "+" + "+".join("─" * (width + 2) for width in col_widths) + "+\n"


            # 데이터 행들
            for row in table_data:
                output_text += "|"
                for col_idx, width in enumerate(col_widths):
                    content = row[col_idx] if col_idx < len(row) else ""
                    content = content[:width]  # 너비 제한
                    padding = width - len(content)
                    output_text += f" {content}{' ' * padding} |"
                output_text += "\n"


                # 첫 번째 행 후 구분선 (헤더가 있는 경우)
                if row == table_data[0] and len(table_data) > 1:
                    output_text += "+" + "+".join("─" * (width + 2) for width in col_widths) + "+\n"


            # 하단 경계
            output_text += "+" + "+".join("─" * (width + 2) for width in col_widths) + "+\n"


            return output_text


        except Exception as e:
            self.logger.warning(f"테이블 변환 중 오류: {e}")
            return "\n[테이블 처리 오류]\n"


    def _clean_whitespace(self, text):
        """공백 및 개행 정리"""
        # 탭을 공백으로 변환
        text = text.replace('\t', ' ')


        # 연속된 공백을 단일 공백으로
        import re
        text = re.sub(r' +', ' ', text)


        # 줄 끝 공백 제거
        lines = text.split('\n')
        lines = [line.rstrip() for line in lines]
        text = '\n'.join(lines)


        return text


    def _remove_empty_lines(self, text):
        """빈 줄 제거"""
        lines = text.split('\n')
        non_empty_lines = [line for line in lines if line.strip()]
        return '\n'.join(non_empty_lines)


    def _clean_special_characters(self, text):
        """특수 문자 정리"""
        # 일반적으로 문제가 되는 특수 문자들 정리
        replacements = {
            '\u00A0': ' ',      # 비표시 공백
            '\u2009': ' ',      # 얇은 공백
            '\u200B': '',       # 영폭 공백
            '\u200C': '',       # 영폭 비결합자
            '\u200D': '',       # 영폭 결합자
            '\uFEFF': '',       # 바이트 순서 표시
            '\u2018': "'",      # 왼쪽 작은따옴표
            '\u2019': "'",      # 오른쪽 작은따옴표
            '\u201C': '"',      # 왼쪽 큰따옴표
            '\u201D': '"',      # 오른쪽 큰따옴표
            '\u2013': '-',      # en dash
            '\u2014': '-',      # em dash
            '\u2026': '...',    # 줄임표
        }


        for old_char, new_char in replacements.items():
            text = text.replace(old_char, new_char)


        return text


    def chunk_text(self, text, max_tokens=None, min_tokens=None, overlap_percentage=None):
        """
        텍스트를 청킹합니다 (백엔드 tools.py의 chunked_texts 함수와 동일)

        Args:
            text: 청킹할 텍스트
            max_tokens: 최대 토큰 수 (기본값: 설정값 사용)
            min_tokens: 최소 토큰 수 (기본값: 설정값 사용)
            overlap_percentage: 중복 비율 (기본값: 설정값 사용)
        """
        # 기본값 설정
        max_tokens = max_tokens or self.config.processing_config['chunk_max_tokens']
        min_tokens = min_tokens or self.config.processing_config['chunk_min_tokens']
        overlap_percentage = overlap_percentage or self.config.processing_config['chunk_overlap_percentage']


        # 토큰화
        tokens = self.tokenizer.encode(text)
        overlap_tokens = int(max_tokens * overlap_percentage)


        chunk_list = []
        start_index = 0


        while start_index < len(tokens):
            if start_index + max_tokens >= len(tokens):  # 마지막 청크인 경우
                remaining_tokens = len(tokens) - start_index
                if remaining_tokens < min_tokens:  # 마지막 청크가 너무 작은 경우
                    if chunk_list:  # 이전 청크가 있다면 합치기
                        chunk_list[-1] += self.tokenizer.decode(tokens[start_index:])
                        break
                    else:  # 청크가 하나도 없다면 그대로 사용
                        chunk_list.append(self.tokenizer.decode(tokens[start_index:]))
                        break
                else:  # 마지막 청크가 충분히 큰 경우
                    chunk_list.append(self.tokenizer.decode(tokens[start_index:]))
                    break
            else:
                end_index = min(start_index + max_tokens, len(tokens))
                chunk_tokens = tokens[start_index:end_index]
                chunk = self.tokenizer.decode(chunk_tokens)
                chunk_list.append(chunk)
                start_index += max_tokens - overlap_tokens


        return chunk_list


    def calculate_tokens(self, text):
        """텍스트의 토큰 수를 계산합니다"""
        return len(self.tokenizer.encode(text))


    def preprocess_document(self, extraction_result):
        """
        전체 문서 전처리를 수행합니다.

        Args:
            extraction_result: TextExtractor에서 반환된 결과
        """
        if not extraction_result.get('success') or not extraction_result.get('text'):
            return {
                'success': False,
                'error': '유효한 추출 결과가 없습니다.'
            }


        import time
        start_time = time.time()


        try:
            # 1. 텍스트 전처리
            preprocessing_result = self.preprocess_text(extraction_result['text'])
            processed_text = preprocessing_result['processed_text']


            # 2. 토큰 수 계산
            token_count = self.calculate_tokens(processed_text)


            # 3. 텍스트 청킹
            chunks = self.chunk_text(processed_text)


            # 4. 청크별 토큰 수 계산
            chunk_tokens = [self.calculate_tokens(chunk) for chunk in chunks]


            # 결과 구성
            result = {
                'success': True,
                'original_extraction': extraction_result,
                'preprocessing': preprocessing_result,
                'processed_text': processed_text,
                'token_count': token_count,
                'chunks': chunks,
                'chunk_count': len(chunks),
                'chunk_tokens': chunk_tokens,
                'processing_time': time.time() - start_time,
                'metadata': {
                    'file_path': extraction_result.get('file_path'),
                    'file_type': extraction_result.get('file_type'),
                    'extraction_method': extraction_result.get('extraction_method'),
                    'preprocessing_steps': preprocessing_result.get('processing_steps', []),
                    'compression_ratio': preprocessing_result.get('compression_ratio', 1.0)
                }
            }


            return result


        except Exception as e:
            self.logger.error(f"문서 전처리 실패: {e}")
            return {
                'success': False,
                'error': str(e),
                'original_extraction': extraction_result
            }


# 전처리기 인스턴스 생성
preprocessor = DocumentPreprocessor(config)


print("🧹 문서 전처리 기능 초기화 완료!", flush=True)
print("=" * 50, flush=True)


print("📋 전처리 기능:", flush=True)
print("   ✨ 유니코드 정규화", flush=True)
print("   🏷️ HTML/XML 태그 정제 (백엔드 tools.py 기반)", flush=True)
print("   📝 공백 및 개행 정리", flush=True)
print("   🗑️ 빈 줄 제거", flush=True)
print("   🔤 특수 문자 정규화", flush=True)
print("   ✂️ 스마트 텍스트 청킹", flush=True)


# 안전한 출력(기본값 포함)
cfg = getattr(config, 'processing_config', {}) if 'config' in globals() else {}
max_tokens = int(cfg.get('chunk_max_tokens', 512) or 512)
min_tokens = int(cfg.get('chunk_min_tokens', 128) or 128)
overlap_percentage = float(cfg.get('chunk_overlap_percentage', 0.1) or 0.1)


print(f"\n⚙️ 청킹 설정:", flush=True)
print(f"   최대 토큰: {max_tokens:,}", flush=True)
print(f"   최소 토큰: {min_tokens:,}", flush=True)
print(f"   중복 비율: {overlap_percentage*100:.1f}%", flush=True)


print(f"\n💡 사용법:", flush=True)
print(f"   - preprocessor.preprocess_text(text): 텍스트만 전처리", flush=True)
print(f"   - preprocessor.chunk_text(text): 텍스트 청킹", flush=True)
print(f"   - preprocessor.preprocess_document(extraction_result): 전체 문서 전처리", flush=True)

🔄 토크나이저 초기화 중...
✅ 토크나이저 초기화 완료!
🧹 문서 전처리 기능 초기화 완료!
📋 전처리 기능:
   ✨ 유니코드 정규화
   🏷️ HTML/XML 태그 정제 (백엔드 tools.py 기반)
   📝 공백 및 개행 정리
   🗑️ 빈 줄 제거
   🔤 특수 문자 정규화
   ✂️ 스마트 텍스트 청킹

⚙️ 청킹 설정:


AttributeError: 'DocumentExtractionTestConfig' object has no attribute 'processing_config'

## ? 최종 단계: 문서 타입별 텍스트 추출 테스트

이 단계에서는 다양한 문서 형식에서 텍스트를 추출하고 저장하여 다음 파이프라인과 연결할 수 있도록 합니다.

In [ ]:
# 📝 문서 타입별 텍스트 추출 테스트


from pathlib import Path
import time
from datetime import datetime


def _get_output_dir():
    try:
        od = getattr(config, 'extracted_text_dir', None)
        if od:
            Path(od).mkdir(parents=True, exist_ok=True)
            return Path(od)
    except Exception:
        pass
    od = Path("/home/admin/wkms-aws/jupyter_notebook/data/opensource_output/extracted_texts")
    od.mkdir(parents=True, exist_ok=True)
    return od


def _get_text_extractor():
    try:
        if 'text_extractor' in globals() and text_extractor:
            return text_extractor
    except Exception:
        pass
    try:
        return DocumentTextExtractor(config)  # type: ignore
    except NameError:
        return TextExtractor(config)  # type: ignore


def _get_document_loader():
    try:
        if 'document_loader' in globals() and document_loader:
            return document_loader
    except Exception:
        pass
    try:
        return DocumentTestLoader(config)  # type: ignore
    except NameError:
        return None


def _safe_doc_info_from_fs(file_path: Path):
    try:
        file_type = getattr(config, 'get_file_type', lambda p: p.suffix.lstrip('.').lower() or 'unknown')(file_path)
    except Exception:
        file_type = file_path.suffix.lstrip('.').lower() or 'unknown'
    size_mb = 0.0
    try:
        size_mb = file_path.stat().st_size/1024/1024
    except Exception:
        pass
    return {'success': True, 'file_type': file_type, 'file_size_mb': size_mb}


def test_file_extraction(file_path, method='auto'):
    """개별 파일의 텍스트 추출 테스트"""
    file_path = str(file_path)
    print(f"🔍 텍스트 추출 테스트: {Path(file_path).name}", flush=True)
    print("=" * 60, flush=True)


    start_time = time.time()


    try:
        # 1단계: 문서 로딩
        print("📄 1단계: 문서 로딩", flush=True)
        loader = _get_document_loader()
        fp = Path(file_path)
        if loader is None:
            print("⚠️ DocumentLoader를 사용할 수 없어 파일 타입만 추정합니다.", flush=True)
            doc_info = _safe_doc_info_from_fs(fp)
        else:
            # 최신 API 이름으로 시도하고, 없으면 폴백
            if hasattr(loader, 'load_document_info'):
                doc_info = loader.load_document_info(file_path)  # type: ignore
            else:
                print("⚠️ load_document_info가 없어 파일 시스템 기반으로 기본 정보만 사용합니다.", flush=True)
                doc_info = _safe_doc_info_from_fs(fp)
        if not doc_info or not doc_info.get('success', False):
            print(f"❌ 문서 로딩 실패: {doc_info.get('error', 'Unknown error') if isinstance(doc_info, dict) else 'Unknown error'}", flush=True)
            return None


        print(f"   ✅ 파일 타입: {doc_info.get('file_type', 'unknown')}", flush=True)
        print(f"   📊 파일 크기: {doc_info.get('file_size_mb', 0):.2f} MB", flush=True)


        # 2단계: 텍스트 추출
        print(f"\n🔤 2단계: 텍스트 추출 (방법: {method})", flush=True)
        extractor = _get_text_extractor()
        extraction_result = extractor.extract_text(file_path, method=method)
        if not extraction_result.get('success', False):
            print(f"❌ 텍스트 추출 실패: {extraction_result.get('error', 'Unknown error')}", flush=True)
            return None


        extracted_text = extraction_result.get('text', '')
        metadata = extraction_result.get('metadata', {})


        print(f"   ✅ 추출 완료: {len(extracted_text):,}자", flush=True)
        print(f"   📊 추출 방법: {extraction_result.get('extraction_method', method)}", flush=True)
        print(f"   ⏱️ 처리 시간: {extraction_result.get('processing_time', 0):.2f}초", flush=True)


        # 메타데이터 표시
        if metadata:
            print(f"   📋 메타데이터:", flush=True)
            for key, value in metadata.items():
                if key not in ['page_results']:  # 너무 긴 데이터 제외
                    print(f"      {key}: {value}", flush=True)


        # 3단계: 텍스트 저장
        print(f"\n💾 3단계: 추출된 텍스트 저장", flush=True)
        file_stem = Path(file_path).stem
        save_result = save_extracted_text(extracted_text, file_stem, extraction_result)


        if save_result:
            print(f"   ✅ 저장 완료: {save_result}", flush=True)


        # 4단계: 텍스트 미리보기
        print(f"\n👀 4단계: 텍스트 미리보기 (처음 500자)", flush=True)
        print("-" * 60, flush=True)
        preview_text = extracted_text[:500] + "..." if len(extracted_text) > 500 else extracted_text
        print(preview_text, flush=True)
        print("-" * 60, flush=True)


        total_time = time.time() - start_time
        print(f"\n✅ 전체 테스트 완료! 총 소요시간: {total_time:.2f}초", flush=True)


        return {
            'success': True,
            'file_path': file_path,
            'file_type': doc_info.get('file_type', 'unknown'),
            'extraction_method': extraction_result.get('extraction_method', method),
            'text_length': len(extracted_text),
            'processing_time': total_time,
            'extracted_text': extracted_text,
            'metadata': metadata,
            'saved_file': str(save_result) if save_result else None
        }


    except Exception as e:
        print(f"❌ 테스트 실패: {e}", flush=True)
        return {
            'success': False,
            'error': str(e),
            'file_path': file_path
        }


def save_extracted_text(text, file_stem, extraction_result):
    """추출된 텍스트를 파일로 저장"""
    try:
        output_dir = _get_output_dir()
        output_file = output_dir / f"{file_stem}_extracted.txt"


        # 헤더 정보 생성
        header = f"=== 추출된 텍스트 ===\n"
        header += f"원본 파일: {file_stem}\n"
        header += f"추출 방법: {extraction_result.get('extraction_method', 'unknown')}\n"
        header += f"추출 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
        header += f"텍스트 길이: {len(text):,}자\n"
        header += f"처리 시간: {extraction_result.get('processing_time', 0):.2f}초\n"
        header += "=" * 80 + "\n\n"


        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(header + text)


        return output_file
    except Exception as e:
        print(f"⚠️ 텍스트 저장 실패: {e}", flush=True)
        return None


print("📝 문서 타입별 텍스트 추출 테스트 기능 준비 완료!", flush=True)
print("💡 사용법: test_file_extraction('파일경로', method='auto')", flush=True)
print("📊 지원 방법: 'auto', 'pypdf2', 'pdfplumber', 'ocr'", flush=True)
try:
    print(f"💾 저장 경로: {_get_output_dir()}", flush=True)
except Exception:
    pass

📝 문서 타입별 텍스트 추출 테스트 기능 준비 완료!
💡 사용법: test_file_extraction('파일경로', method='auto')
📊 지원 방법: 'auto', 'pypdf2', 'pdfplumber', 'ocr'
💾 저장 경로: /home/admin/wkms-aws/jupyter_notebook/data/opensource_output/extracted_texts
💡 사용법: test_file_extraction('파일경로', method='auto')
📊 지원 방법: 'auto', 'pypdf2', 'pdfplumber', 'ocr'
💾 저장 경로: /home/admin/wkms-aws/jupyter_notebook/data/opensource_output/extracted_texts


In [19]:
# 🚀 테스트 파일 실행 예시


from pathlib import Path


def _safe_get_file_type(p: Path) -> str:
    try:
        if 'config' in globals() and hasattr(config, 'get_file_type'):
            return config.get_file_type(p)  # type: ignore
    except Exception:
        pass
    return (p.suffix.lstrip('.').lower() or 'unknown')


def _resolve_sample_files(max_count: int = 10):
    # 1) 이미 수집된 available_files 변수가 있으면 사용
    try:
        if 'available_files' in globals() and available_files:
            return [Path(f) for f in available_files[:max_count]]
    except Exception:
        pass


    # 2) document_loader가 있다면 그 결과를 활용
    try:
        if 'document_loader' in globals() and document_loader:
            files = []
            # document_loader가 스캔 결과를 가지고 있을 수 있으므로 최대한 활용
            if hasattr(document_loader, 'files_by_type') and document_loader.files_by_type:
                for lst in document_loader.files_by_type.values():
                    files.extend(lst)
            elif hasattr(document_loader, 'available_files') and document_loader.available_files:
                files.extend(document_loader.available_files)
            files = [Path(f) for f in files if f]
            if files:
                return files[:max_count]
    except Exception:
        pass


    # 3) 입력 디렉토리에서 직접 스캔
    try:
        input_dir = None
        try:
            input_dir = Path(getattr(config, 'input_docs_dir', None) or "/home/admin/wkms-aws/jupyter_notebook/data/input_docs")
        except Exception:
            input_dir = Path("/home/admin/wkms-aws/jupyter_notebook/data/input_docs")
        patterns = ['*.pdf','*.docx','*.pptx','*.txt','*.html','*.xlsx','*.csv']
        files = []
        if input_dir.exists():
            for pat in patterns:
                files.extend(input_dir.glob(pat))
        return files[:max_count] if files else []
    except Exception:
        return []


# 샘플 파일 수집
sample_files = _resolve_sample_files(max_count=10)


print("📁 사용 가능한 테스트 파일:", flush=True)
if not sample_files:
    print("   ⚠️ 샘플 파일을 찾지 못했습니다. input_docs 디렉토리를 확인하세요.", flush=True)
else:
    for i, file_path in enumerate(sample_files, 1):
        file_info = f"{i}. {Path(file_path).name} ({_safe_get_file_type(Path(file_path))})"
        print(f"   {file_info}", flush=True)


print(f"\n💡 테스트 실행 방법:", flush=True)
print("# 특정 파일 테스트", flush=True)
print("test_result = test_file_extraction(sample_files[0])  # 첫 번째 파일", flush=True)
print("")
print("# OCR 강제 사용 (이미지 기반 PDF용)", flush=True)
print("test_result = test_file_extraction(sample_files[0], method='ocr')", flush=True)
print("")
print("# 모든 파일 테스트", flush=True)
print("for file_path in sample_files[:3]:  # 처음 3개 파일만", flush=True)
print("    test_file_extraction(file_path)", flush=True)


print(f"\n📂 결과 저장 경로:", flush=True)
try:
    out_dir = _get_output_dir()
    print(f"   📄 추출된 텍스트: {out_dir}", flush=True)
    print(f"   📊 OCR 상세 결과: {out_dir} (OCR 사용 시)", flush=True)
except Exception:
    try:
        print(f"   📄 추출된 텍스트: {config.extracted_text_dir}", flush=True)
    except Exception:
        print(f"   📄 추출된 텍스트: /home/admin/wkms-aws/jupyter_notebook/data/opensource_output/extracted_texts", flush=True)


print(f"\n🔗 다음 단계 연결:", flush=True)
print("   1. 추출된 텍스트 파일을 백엔드 tools.py로 전달", flush=True)
print("   2. chunked_texts() 함수로 청킹 처리", flush=True)
print("   3. 임베딩 생성 및 벡터스토어 저장", flush=True)

📁 사용 가능한 테스트 파일:
   1. 20_ProductSpec_SmartInsulinPump_KO_v0.1.pdf (pdf)
   2. test1.pdf (pdf)
   3. test.pdf (pdf)

💡 테스트 실행 방법:
# 특정 파일 테스트
test_result = test_file_extraction(sample_files[0])  # 첫 번째 파일

# OCR 강제 사용 (이미지 기반 PDF용)
   1. 20_ProductSpec_SmartInsulinPump_KO_v0.1.pdf (pdf)
   2. test1.pdf (pdf)
   3. test.pdf (pdf)

💡 테스트 실행 방법:
# 특정 파일 테스트
test_result = test_file_extraction(sample_files[0])  # 첫 번째 파일

# OCR 강제 사용 (이미지 기반 PDF용)
test_result = test_file_extraction(sample_files[0], method='ocr')

# 모든 파일 테스트
for file_path in sample_files[:3]:  # 처음 3개 파일만
    test_file_extraction(file_path)

📂 결과 저장 경로:
test_result = test_file_extraction(sample_files[0], method='ocr')

# 모든 파일 테스트
for file_path in sample_files[:3]:  # 처음 3개 파일만
    test_file_extraction(file_path)

📂 결과 저장 경로:
   📄 추출된 텍스트: /home/admin/wkms-aws/jupyter_notebook/data/opensource_output/extracted_texts
   📊 OCR 상세 결과: /home/admin/wkms-aws/jupyter_notebook/data/opensource_output/extracted_texts (OCR 사용 시)

🔗 다음 

In [20]:
# 🧪 test.pdf 파일 텍스트 추출 테스트


from pathlib import Path
import time


def _ensure_path(p: str | Path) -> Path:
    return p if isinstance(p, Path) else Path(p)


test_pdf_path = _ensure_path("/home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf")


print("🔍 test.pdf 파일 텍스트 추출 테스트 시작", flush=True)
print("=" * 80, flush=True)


# 파일 존재 여부 확인
if not test_pdf_path.exists():
    print(f"❌ 파일을 찾을 수 없습니다: {test_pdf_path}", flush=True)
    print(f"📂 현재 input_docs 디렉토리 내용:", flush=True)
    input_docs_dir = test_pdf_path.parent
    if input_docs_dir.exists():
        for file in input_docs_dir.iterdir():
            print(f"   📄 {file.name}", flush=True)
    else:
        print(f"   📂 디렉토리가 존재하지 않습니다: {input_docs_dir}", flush=True)
else:
    print(f"✅ 파일 확인됨: {test_pdf_path.name}", flush=True)
    try:
        size_mb = test_pdf_path.stat().st_size / 1024 / 1024
        print(f"📊 파일 크기: {size_mb:.2f} MB", flush=True)
    except Exception:
        pass
    
    # 1단계: 자동 방법으로 텍스트 추출 테스트
    print(f"\n🚀 1단계: 자동 방법으로 텍스트 추출", flush=True)
    auto_result = test_file_extraction(str(test_pdf_path), method='auto')
    
    if auto_result and auto_result.get('success'):
        print(f"\n📊 자동 추출 결과:", flush=True)
        print(f"   📄 추출 방법: {auto_result.get('extraction_method')}", flush=True)
        print(f"   📝 텍스트 길이: {auto_result.get('text_length', 0):,}자", flush=True)
        print(f"   ⏱️ 처리 시간: {auto_result.get('processing_time', 0.0):.2f}초", flush=True)
        
        # 텍스트가 충분히 추출되었는지 확인
        if auto_result.get('text_length', 0) < 100:
            print(f"\n⚠️ 추출된 텍스트가 적습니다. 이미지 기반 PDF일 가능성이 있습니다.", flush=True)
            print(f"🔄 2단계: OCR 방법으로 재시도", flush=True)
            
            ocr_result = test_file_extraction(str(test_pdf_path), method='ocr')
            
            if ocr_result and ocr_result.get('success'):
                print(f"\n📊 OCR 추출 결과:", flush=True)
                print(f"   📄 추출 방법: {ocr_result.get('extraction_method')}", flush=True)
                print(f"   📝 텍스트 길이: {ocr_result.get('text_length', 0):,}자", flush=True)
                print(f"   ⏱️ 처리 시간: {ocr_result.get('processing_time', 0.0):.2f}초", flush=True)
                
                # 결과 비교
                auto_len = auto_result.get('text_length', 0)
                ocr_len = ocr_result.get('text_length', 0)
                improve = ((ocr_len - auto_len) / max(auto_len, 1) * 100)
                print(f"\n📈 결과 비교:", flush=True)
                print(f"   자동 방법: {auto_len:,}자", flush=True)
                print(f"   OCR 방법: {ocr_len:,}자", flush=True)
                print(f"   개선율: {improve:+.1f}%", flush=True)
        else:
            print(f"\n✅ 자동 방법으로 충분한 텍스트 추출 완료!", flush=True)
    
    print(f"\n💾 저장된 결과 파일:", flush=True)
    try:
        out_dir = _get_output_dir()
    except Exception:
        try:
            out_dir = Path(config.extracted_text_dir)
        except Exception:
            out_dir = Path("/home/admin/wkms-aws/jupyter_notebook/data/opensource_output/extracted_texts")
    try:
        extracted_files = list(out_dir.glob("test*"))
        for file in sorted(extracted_files):
            print(f"   📄 {file.name}", flush=True)
    except Exception:
        pass
    
    print(f"\n🔗 다음 단계:", flush=True)
    print(f"   1. 추출된 텍스트를 백엔드 tools.py로 전달", flush=True)
    print(f"   2. chunked_texts() 함수로 청킹 처리", flush=True)
    print(f"   3. Azure OpenAI로 임베딩 생성", flush=True)
    print(f"   4. 벡터스토어에 저장", flush=True)

🔍 test.pdf 파일 텍스트 추출 테스트 시작
✅ 파일 확인됨: test.pdf
📊 파일 크기: 11.85 MB

🚀 1단계: 자동 방법으로 텍스트 추출
🔍 텍스트 추출 테스트: test.pdf
📄 1단계: 문서 로딩
❌ 테스트 실패: 'DocumentTestLoader' object has no attribute 'load_document'

💾 저장된 결과 파일:

🔗 다음 단계:
   1. 추출된 텍스트를 백엔드 tools.py로 전달
   2. chunked_texts() 함수로 청킹 처리
   3. Azure OpenAI로 임베딩 생성
   4. 벡터스토어에 저장
✅ 파일 확인됨: test.pdf
📊 파일 크기: 11.85 MB

🚀 1단계: 자동 방법으로 텍스트 추출
🔍 텍스트 추출 테스트: test.pdf
📄 1단계: 문서 로딩
❌ 테스트 실패: 'DocumentTestLoader' object has no attribute 'load_document'

💾 저장된 결과 파일:

🔗 다음 단계:
   1. 추출된 텍스트를 백엔드 tools.py로 전달
   2. chunked_texts() 함수로 청킹 처리
   3. Azure OpenAI로 임베딩 생성
   4. 벡터스토어에 저장


In [ ]:
# 🔧 test.pdf 문제 진단 및 직접 테스트


from pathlib import Path
from datetime import datetime


def _lazy_import_pypdf2():
    try:
        import PyPDF2
        return PyPDF2
    except Exception as e:
        print(f"      ⚠️ PyPDF2 import 실패: {e}", flush=True)
        return None


def _lazy_import_pdfplumber():
    try:
        import pdfplumber
        return pdfplumber
    except Exception as e:
        print(f"      ⚠️ pdfplumber import 실패: {e}", flush=True)
        return None


test_pdf_path = Path("/home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf")


print("🔧 test.pdf 문제 진단", flush=True)
print("=" * 60, flush=True)


# 1단계: 파일 기본 정보 확인
print("📂 1단계: 파일 기본 정보", flush=True)
if test_pdf_path.exists():
    try:
        stat = test_pdf_path.stat()
        print(f"   ✅ 파일 존재: {test_pdf_path.name}", flush=True)
        print(f"   📊 크기: {stat.st_size / 1024 / 1024:.2f} MB", flush=True)
        print(f"   🕒 수정일: {datetime.fromtimestamp(stat.st_mtime)}", flush=True)
    except Exception as e:
        print(f"   ⚠️ 파일 정보 조회 실패: {e}", flush=True)
    
    # 2단계: DocumentLoader 직접 테스트
    print(f"\n📄 2단계: DocumentLoader 직접 테스트", flush=True)
    try:
        if 'document_loader' in globals() and document_loader:
            doc_info = document_loader.load_document(str(test_pdf_path))  # type: ignore
            print(f"   📋 반환된 키들: {list(doc_info.keys())}", flush=True)
            
            if 'success' in doc_info:
                if doc_info['success']:
                    print(f"   ✅ 로딩 성공", flush=True)
                    print(f"   📄 파일 타입: {doc_info.get('file_type', 'N/A')}", flush=True)
                    print(f"   📊 파일 크기: {doc_info.get('file_size_mb', 'N/A')} MB", flush=True)
                    print(f"   📝 내용 길이: {len(doc_info.get('file_content', ''))}", flush=True)
                else:
                    print(f"   ❌ 로딩 실패: {doc_info.get('error', 'Unknown error')}", flush=True)
            else:
                print(f"   ⚠️ 'success' 키가 없음 - 반환 구조 문제", flush=True)
        else:
            print(f"   ⚠️ document_loader 가 초기화되지 않음", flush=True)
            
    except Exception as e:
        print(f"   ❌ DocumentLoader 오류: {e}", flush=True)
        print(f"   🔍 오류 상세: {type(e).__name__}", flush=True)
    
    # 3단계: TextExtractor 직접 테스트
    print(f"\n🔤 3단계: TextExtractor 직접 테스트", flush=True)
    try:
        if 'text_extractor' not in globals() or not text_extractor:
            # fallback 생성
            try:
                extractor = DocumentTextExtractor(config)  # type: ignore
            except NameError:
                extractor = TextExtractor(config)  # type: ignore
        else:
            extractor = text_extractor
        
        # 각 방법별로 개별 테스트
        methods = ['pypdf2', 'pdfplumber']
        
        for method in methods:
            print(f"\n   🔄 {method} 방법 테스트:", flush=True)
            try:
                result = extractor.extract_text(str(test_pdf_path), method=method)
                
                if result.get('success', False):
                    text_len = len(result.get('text', ''))
                    print(f"      ✅ 성공: {text_len:,}자", flush=True)
                    if text_len > 0:
                        preview = result['text'][:100].replace('\n', ' ')
                        print(f"      👀 미리보기: {preview}...", flush=True)
                else:
                    print(f"      ❌ 실패: {result.get('error', 'Unknown error')}", flush=True)
                    
            except Exception as e:
                print(f"      ❌ 오류: {e}", flush=True)
    
    except Exception as e:
        print(f"   ❌ TextExtractor 전체 오류: {e}", flush=True)
        
    # 4단계: PDF 파일 직접 읽기 테스트
    print(f"\n📖 4단계: PDF 라이브러리 직접 테스트", flush=True)
    
    # PyPDF2 테스트
    try:
        print(f"   📚 PyPDF2 테스트:", flush=True)
        PyPDF2 = _lazy_import_pypdf2()
        if PyPDF2 is None:
            raise RuntimeError("PyPDF2를 사용할 수 없습니다.")
        with open(test_pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            page_count = len(pdf_reader.pages)
            print(f"      📄 페이지 수: {page_count}", flush=True)
            
            if page_count > 0:
                try:
                    first_page_text = pdf_reader.pages[0].extract_text() or ""
                except Exception:
                    first_page_text = ""
                print(f"      📝 첫 페이지 텍스트 길이: {len(first_page_text)}", flush=True)
                if len(first_page_text) > 0:
                    preview = first_page_text[:100].replace('\n', ' ')
                    print(f"      👀 첫 페이지 미리보기: {preview}...", flush=True)
                else:
                    print(f"      ⚠️ 첫 페이지에서 텍스트를 추출할 수 없음", flush=True)
    except Exception as e:
        print(f"      ❌ PyPDF2 오류: {e}", flush=True)
    
    # pdfplumber 테스트
    try:
        print(f"\n   📊 pdfplumber 테스트:", flush=True)
        pdfplumber = _lazy_import_pdfplumber()
        if pdfplumber is None:
            raise RuntimeError("pdfplumber를 사용할 수 없습니다.")
        with pdfplumber.open(test_pdf_path) as pdf:
            page_count = len(pdf.pages)
            print(f"      📄 페이지 수: {page_count}", flush=True)
            
            if page_count > 0:
                first_page_text = pdf.pages[0].extract_text() or ""
                print(f"      📝 첫 페이지 텍스트 길이: {len(first_page_text)}", flush=True)
                if len(first_page_text) > 0:
                    preview = first_page_text[:100].replace('\n', ' ')
                    print(f"      👀 첫 페이지 미리보기: {preview}...", flush=True)
                else:
                    print(f"      ⚠️ 첫 페이지에서 텍스트를 추출할 수 없음 (이미지 기반 PDF일 가능성)", flush=True)
    except Exception as e:
        print(f"      ❌ pdfplumber 오류: {e}", flush=True)


else:
    print(f"❌ 파일이 존재하지 않습니다: {test_pdf_path}", flush=True)
    # 대안 파일 경로 확인
    possible_paths = [
        Path("/home/admin/wkms-aws/jupyter_notebook/data/input_docs"),
        Path("/home/admin/wkms-aws/data/input_docs"),
        Path("/home/admin/wkms-aws")
    ]
    
    print(f"\n🔍 대안 경로 탐색:", flush=True)
    for path in possible_paths:
        try:
            if path.exists():
                print(f"   📂 {path}:", flush=True)
                pdf_files = list(path.glob("*.pdf"))
                if pdf_files:
                    for pdf_file in pdf_files[:5]:  # 최대 5개만 표시
                        print(f"      📄 {pdf_file.name}", flush=True)
                else:
                    print(f"      📄 PDF 파일 없음", flush=True)
            else:
                print(f"   ❌ 존재하지 않음: {path}", flush=True)
        except Exception:
            pass


print(f"\n🎯 진단 완료!", flush=True)
print(f"💡 위 결과를 바탕으로 문제를 해결하겠습니다.", flush=True)

In [ ]:
# 🧩 배치 테스트: 다양한 샘플 파일 일괄 처리


from pathlib import Path
import time


def batch_test(files: list[Path] | None = None, limit: int = 5, method: str = 'auto'):

    print("? 배치 테스트 시작", flush=True)

    start = time.time()

    if files is None:

        try:

            files = _resolve_sample_files(max_count=limit)

        except Exception:

            files = []

    files = [Path(f) for f in files][:limit]

    if not files:

        print("⚠️ 테스트할 파일이 없습니다.", flush=True)

        return {"success": False, "count": 0, "results": []}



    results = []

    for i, f in enumerate(files, 1):

        print(f"\n[{i}/{len(files)}] {f.name} 처리", flush=True)

        try:

            r = test_file_extraction(str(f), method=method)

            results.append(r)

        except Exception as e:

            print(f"❌ 처리 실패: {e}", flush=True)

            results.append({"success": False, "error": str(e), "file": str(f)})



    ok = sum(1 for r in results if r and r.get('success'))

    elapsed = time.time() - start

    print(f"\n✅ 배치 완료: {ok}/{len(results)} 성공, {elapsed:.2f}s", flush=True)

    return {"success": True, "count": len(results), "ok": ok, "elapsed": elapsed, "results": results}



print("💡 사용법: batch_test(limit=3, method='auto')", flush=True)

In [ ]:
# 🔍 코드 점검 및 주요 문제 해결

print("🔍 노트북 코드 점검 및 주요 문제 진단")
print("=" * 70)

# 1. 필수 변수들이 제대로 생성되었는지 확인
print("📋 1단계: 핵심 객체 상태 확인")

# config 객체 확인
try:
    print(f"   ✅ config: {type(config).__name__}")
    print(f"      📂 extracted_text_dir: {config.extracted_text_dir}")
    print(f"      🔧 OCR 활성화: {config.processing_config.get('ocr_enabled', False)}")
except NameError:
    print(f"   ❌ config 객체가 없습니다!")

# document_loader 확인
try:
    print(f"   ✅ document_loader: {type(document_loader).__name__}")
except NameError:
    print(f"   ❌ document_loader 객체가 없습니다!")

# text_extractor 확인
try:
    print(f"   ✅ text_extractor: {type(text_extractor).__name__}")
except NameError:
    print(f"   ❌ text_extractor 객체가 없습니다!")

# 2. 주요 라이브러리 임포트 상태 확인
print(f"\n📚 2단계: 필수 라이브러리 임포트 확인")

libraries_to_check = [
    ('PyPDF2', 'PyPDF2'),
    ('pdfplumber', 'pdfplumber'), 
    ('docx', 'python-docx'),
    ('pptx', 'python-pptx'),
    ('pytesseract', 'pytesseract'),
    ('PIL', 'Pillow'),
    ('pdf2image', 'pdf2image'),
    ('pandas', 'pandas')
]

missing_libraries = []
for lib_name, package_name in libraries_to_check:
    try:
        __import__(lib_name)
        print(f"   ✅ {lib_name}")
    except ImportError:
        print(f"   ❌ {lib_name} (패키지: {package_name})")
        missing_libraries.append(package_name)

if missing_libraries:
    print(f"\n⚠️ 누락된 라이브러리: {missing_libraries}")
    print(f"💡 설치 명령: pip install {' '.join(missing_libraries)}")

# 3. test_file_extraction 함수 문제 점검
print(f"\n🔧 3단계: test_file_extraction 함수 문제 점검")

# 함수가 정의되어 있는지 확인
try:
    test_file_extraction
    print(f"   ✅ test_file_extraction 함수가 정의됨")
    
    # 함수 내부에서 사용하는 변수들 확인
    print(f"   📋 함수 내부 종속성 확인:")
    
    # document_loader.load_document 메소드 확인
    try:
        if hasattr(document_loader, 'load_document'):
            print(f"      ✅ document_loader.load_document 메소드 존재")
        else:
            print(f"      ❌ document_loader.load_document 메소드 없음")
    except:
        print(f"      ❌ document_loader 접근 불가")
    
    # text_extractor.extract_text 메소드 확인
    try:
        if hasattr(text_extractor, 'extract_text'):
            print(f"      ✅ text_extractor.extract_text 메소드 존재")
        else:
            print(f"      ❌ text_extractor.extract_text 메소드 없음")
    except:
        print(f"      ❌ text_extractor 접근 불가")
        
    # save_extracted_text 함수 확인
    try:
        save_extracted_text
        print(f"      ✅ save_extracted_text 함수 정의됨")
    except NameError:
        print(f"      ❌ save_extracted_text 함수가 정의되지 않음")
        
except NameError:
    print(f"   ❌ test_file_extraction 함수가 정의되지 않음")

# 4. 간단한 PDF 처리 테스트
print(f"\n🧪 4단계: 기본 PDF 처리 능력 테스트")

# 테스트용 간단한 PDF 파일 생성 및 테스트
try:
    # 먼저 test.pdf가 있는지 확인
    test_pdf = Path("/home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf")
    
    if test_pdf.exists():
        print(f"   📄 테스트 대상: {test_pdf.name} ({test_pdf.stat().st_size / 1024 / 1024:.2f} MB)")
        
        # PyPDF2로 직접 테스트
        try:
            import PyPDF2
            with open(test_pdf, 'rb') as f:
                reader = PyPDF2.PdfReader(f)
                print(f"      📚 PyPDF2: {len(reader.pages)}페이지")
                
                if len(reader.pages) > 0:
                    first_page_text = reader.pages[0].extract_text()
                    print(f"      📝 첫 페이지 텍스트: {len(first_page_text)}자")
                    
                    if len(first_page_text.strip()) == 0:
                        print(f"      ⚠️ 이미지 기반 PDF - OCR 필요")
                    else:
                        preview = first_page_text.strip()[:100].replace('\n', ' ')
                        print(f"      👀 미리보기: {preview}...")
                        
        except Exception as e:
            print(f"      ❌ PyPDF2 직접 테스트 실패: {e}")
            
    else:
        print(f"   ❌ 테스트 파일 없음: {test_pdf}")
        
        # 대안 파일 찾기
        input_dir = test_pdf.parent
        if input_dir.exists():
            pdf_files = list(input_dir.glob("*.pdf"))
            if pdf_files:
                print(f"   📂 사용 가능한 PDF 파일들:")
                for pdf_file in pdf_files[:3]:
                    print(f"      📄 {pdf_file.name}")
            else:
                print(f"   📂 PDF 파일이 없음: {input_dir}")
        
except Exception as e:
    print(f"   ❌ PDF 테스트 실패: {e}")

print(f"\n🎯 점검 완료!")
print(f"📋 다음 셀에서 발견된 문제들을 수정하겠습니다.")

In [ ]:
# 🛠️ 텍스트 추출 문제 수정 및 강화

print("🛠️ 텍스트 추출 문제 수정 중...")
print("=" * 70)

# 1. 안전한 텍스트 추출 함수 재정의
def safe_extract_text_from_pdf(file_path, method='auto'):
    """안전한 PDF 텍스트 추출 함수"""
    
    file_path = Path(file_path)
    
    if not file_path.exists():
        return {
            'success': False,
            'error': f'파일이 존재하지 않습니다: {file_path}',
            'text': '',
            'method': method
        }
    
    print(f"📄 PDF 텍스트 추출: {file_path.name}")
    start_time = time.time()
    
    # 방법별 추출 시도
    if method == 'auto' or method == 'pypdf2':
        try:
            print(f"   🔄 PyPDF2 방법 시도...")
            import PyPDF2
            
            with open(file_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                page_count = len(pdf_reader.pages)
                
                text_parts = []
                for page_num, page in enumerate(pdf_reader.pages, 1):
                    try:
                        page_text = page.extract_text()
                        if page_text.strip():
                            text_parts.append(f"\\n\\n=== 페이지 {page_num} ===\\n{page_text}")
                    except Exception as e:
                        print(f"      ⚠️ 페이지 {page_num} 건너뜀: {e}")
                        continue
                
                full_text = '\\n'.join(text_parts)
                processing_time = time.time() - start_time
                
                print(f"   ✅ PyPDF2 성공: {len(full_text):,}자, {processing_time:.2f}초")
                
                return {
                    'success': True,
                    'text': full_text,
                    'method': 'pypdf2',
                    'pages': page_count,
                    'processing_time': processing_time,
                    'text_length': len(full_text)
                }
                
        except Exception as e:
            print(f"   ❌ PyPDF2 실패: {e}")
            
    if method == 'auto' or method == 'pdfplumber':
        try:
            print(f"   🔄 pdfplumber 방법 시도...")
            import pdfplumber
            
            with pdfplumber.open(file_path) as pdf:
                page_count = len(pdf.pages)
                
                text_parts = []
                for page_num, page in enumerate(pdf.pages, 1):
                    try:
                        page_text = page.extract_text() or ""
                        if page_text.strip():
                            text_parts.append(f"\\n\\n=== 페이지 {page_num} ===\\n{page_text}")
                    except Exception as e:
                        print(f"      ⚠️ 페이지 {page_num} 건너뜀: {e}")
                        continue
                
                full_text = '\\n'.join(text_parts)
                processing_time = time.time() - start_time
                
                print(f"   ✅ pdfplumber 성공: {len(full_text):,}자, {processing_time:.2f}초")
                
                return {
                    'success': True,
                    'text': full_text,
                    'method': 'pdfplumber',
                    'pages': page_count,
                    'processing_time': processing_time,
                    'text_length': len(full_text)
                }
                
        except Exception as e:
            print(f"   ❌ pdfplumber 실패: {e}")
    
    if method == 'ocr':
        try:
            print(f"   🔄 OCR 방법 시도...")
            from pdf2image import convert_from_path
            import pytesseract
            
            # PDF를 이미지로 변환
            images = convert_from_path(file_path, dpi=200)  # 낮은 DPI로 빠른 테스트
            
            text_parts = []
            for page_num, image in enumerate(images, 1):
                try:
                    page_text = pytesseract.image_to_string(image, lang='kor+eng')
                    if page_text.strip():
                        text_parts.append(f"\\n\\n=== 페이지 {page_num} (OCR) ===\\n{page_text}")
                        print(f"      📄 페이지 {page_num}: {len(page_text):,}자")
                except Exception as e:
                    print(f"      ⚠️ 페이지 {page_num} OCR 실패: {e}")
                    continue
            
            full_text = '\\n'.join(text_parts)
            processing_time = time.time() - start_time
            
            print(f"   ✅ OCR 성공: {len(full_text):,}자, {processing_time:.2f}초")
            
            return {
                'success': True,
                'text': full_text,
                'method': 'ocr',
                'pages': len(images),
                'processing_time': processing_time,
                'text_length': len(full_text)
            }
            
        except Exception as e:
            print(f"   ❌ OCR 실패: {e}")
    
    # 모든 방법 실패
    processing_time = time.time() - start_time
    return {
        'success': False,
        'error': '모든 추출 방법이 실패했습니다',
        'text': '',
        'method': method,
        'processing_time': processing_time
    }

# 2. 간단한 텍스트 저장 함수
def simple_save_text(text, filename_prefix, method='unknown'):
    """간단한 텍스트 저장 함수"""
    try:
        # 출력 디렉토리 확인/생성
        if 'config' in globals() and hasattr(config, 'extracted_text_dir'):
            output_dir = config.extracted_text_dir
        else:
            output_dir = Path("/home/admin/wkms-aws/jupyter_notebook/data/opensource_output/extracted_texts")
        
        output_dir.mkdir(parents=True, exist_ok=True)
        
        # 파일 저장
        output_file = output_dir / f"{filename_prefix}_{method}_extracted.txt"
        
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(f"=== 텍스트 추출 결과 ===\\n")
            f.write(f"파일: {filename_prefix}\\n")
            f.write(f"방법: {method}\\n")
            f.write(f"추출 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\\n")
            f.write(f"텍스트 길이: {len(text):,}자\\n")
            f.write("=" * 50 + "\\n\\n")
            f.write(text)
        
        print(f"   💾 저장 완료: {output_file.name}")
        return output_file
        
    except Exception as e:
        print(f"   ❌ 저장 실패: {e}")
        return None

print("✅ 수정 함수들이 정의되었습니다!")
print("💡 다음 셀에서 test.pdf로 테스트를 진행합니다.")

In [ ]:
# 🧪 수정된 함수로 test.pdf 실제 테스트

print("🧪 수정된 함수로 test.pdf 테스트 시작")
print("=" * 80)

# 테스트 파일 경로
test_pdf_path = Path("/home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf")

if test_pdf_path.exists():
    file_size_mb = test_pdf_path.stat().st_size / 1024 / 1024
    print(f"📄 테스트 대상: {test_pdf_path.name}")
    print(f"📊 파일 크기: {file_size_mb:.2f} MB")
    
    # 1단계: 각 방법별로 순차 테스트
    methods_to_test = [
        ('pypdf2', 'PyPDF2 (기본 빠른 방법)'),
        ('pdfplumber', 'pdfplumber (고급 방법)'),
        ('ocr', 'OCR (이미지 기반 PDF용)')
    ]
    
    results = {}
    
    for method, description in methods_to_test:
        print(f"\\n🔄 {description} 테스트")
        print("-" * 50)
        
        result = safe_extract_text_from_pdf(test_pdf_path, method=method)
        results[method] = result
        
        if result['success']:
            # 텍스트 저장
            saved_file = simple_save_text(
                result['text'], 
                test_pdf_path.stem, 
                result['method']
            )
            result['saved_file'] = saved_file
            
            # 텍스트 미리보기
            if len(result['text']) > 0:
                preview = result['text'][:200].replace('\\n', ' ').strip()
                print(f"   👀 미리보기: {preview}...")
            else:
                print(f"   ⚠️ 추출된 텍스트가 없습니다")
        
        # 큰 파일의 경우 OCR은 시간이 많이 걸리므로 조건부 실행
        if method == 'ocr' and file_size_mb > 10:
            user_input = input(f"\\n⚠️ 파일이 {file_size_mb:.1f}MB로 큽니다. OCR을 계속하시겠습니까? (y/n): ")
            if user_input.lower() != 'y':
                print(f"   ⏭️ OCR 테스트를 건너뜁니다")
                results[method] = {
                    'success': False,
                    'error': '사용자가 건너뜀 (파일 크기)',
                    'text': '',
                    'method': method
                }
                continue
    
    # 2단계: 결과 비교 및 요약
    print(f"\\n📊 테스트 결과 요약")
    print("=" * 80)
    
    successful_methods = []
    
    for method, description in methods_to_test:
        if method in results:
            result = results[method]
            
            if result['success']:
                text_len = result.get('text_length', 0)
                proc_time = result.get('processing_time', 0)
                
                print(f"✅ {description}")
                print(f"   📝 추출 텍스트: {text_len:,}자")
                print(f"   ⏱️ 처리 시간: {proc_time:.2f}초")
                print(f"   📄 페이지 수: {result.get('pages', 'N/A')}")
                
                successful_methods.append((method, text_len, proc_time))
                
                if 'saved_file' in result and result['saved_file']:
                    print(f"   💾 저장됨: {result['saved_file'].name}")
                
            else:
                print(f"❌ {description}")
                print(f"   🚫 오류: {result.get('error', 'Unknown error')}")
            
            print()
    
    # 3단계: 최적 방법 추천
    if successful_methods:
        print(f"🏆 최적 방법 추천")
        print("-" * 50)
        
        # 가장 많은 텍스트를 추출한 방법
        best_quality = max(successful_methods, key=lambda x: x[1])
        print(f"🎯 최고 품질: {best_quality[0]} ({best_quality[1]:,}자)")
        
        # 가장 빠른 방법
        best_speed = min(successful_methods, key=lambda x: x[2])
        print(f"⚡ 최고 속도: {best_speed[0]} ({best_speed[2]:.2f}초)")
        
        # 균형잡힌 추천
        if best_quality[1] > 1000:  # 충분한 텍스트가 추출된 경우
            print(f"💡 추천: {best_quality[0]} (충분한 텍스트 추출)")
        elif any(method[0] == 'ocr' for method in successful_methods):
            ocr_result = next(method for method in successful_methods if method[0] == 'ocr')
            if ocr_result[1] > best_quality[1] * 2:  # OCR이 현저히 더 좋은 경우
                print(f"💡 추천: ocr (이미지 기반 PDF - OCR 필수)")
            else:
                print(f"💡 추천: {best_speed[0]} (속도 우선)")
        else:
            print(f"💡 추천: {best_speed[0]} (속도 우선)")
            
    else:
        print(f"❌ 모든 방법이 실패했습니다.")
        print(f"💡 다음을 확인해보세요:")
        print(f"   - 파일이 손상되지 않았는지")
        print(f"   - PDF 파일이 암호화되어 있지 않은지") 
        print(f"   - 필요한 라이브러리가 모두 설치되어 있는지")

else:
    print(f"❌ 테스트 파일이 없습니다: {test_pdf_path}")
    
    # 대안 파일 제안
    input_dir = test_pdf_path.parent
    if input_dir.exists():
        pdf_files = list(input_dir.glob("*.pdf"))
        if pdf_files:
            print(f"\\n📂 사용 가능한 대안 파일들:")
            for i, pdf_file in enumerate(pdf_files[:5], 1):
                size_mb = pdf_file.stat().st_size / 1024 / 1024
                print(f"   {i}. {pdf_file.name} ({size_mb:.2f} MB)")
            
            print(f"\\n💡 대안 파일로 테스트하려면:")
            print(f"   alternative_path = Path('/home/admin/wkms-aws/jupyter_notebook/data/input_docs/파일명.pdf')")
            print(f"   result = safe_extract_text_from_pdf(alternative_path, method='auto')")

print(f"\\n🎯 테스트 완료!")
print(f"📁 추출된 파일들은 다음 경로에 저장됩니다:")
print(f"   /home/admin/wkms-aws/jupyter_notebook/data/opensource_output/extracted_texts/")

In [ ]:
# 📋 일반 PDF 파일 테스트 - ProductSpec_SmartInsulinPump

print("📋 일반 PDF 파일 테스트 시작 - ProductSpec_SmartInsulinPump")
print("=" * 80)

# 테스트 파일 경로
product_pdf_path = Path("/home/admin/wkms-aws/jupyter_notebook/data/input_docs/20_ProductSpec_SmartInsulinPump_KO_v0.1.pdf")

if product_pdf_path.exists():
    file_size_mb = product_pdf_path.stat().st_size / 1024 / 1024
    print(f"📄 테스트 대상: {product_pdf_path.name}")
    print(f"📊 파일 크기: {file_size_mb:.2f} MB")
    
    # 일반 PDF는 텍스트 기반일 가능성이 높으므로 모든 방법 테스트
    methods_to_test = [
        ('pypdf2', 'PyPDF2 (기본 빠른 방법)'),
        ('pdfplumber', 'pdfplumber (고급 방법)'),
        ('ocr', 'OCR (백업 방법)')
    ]
    
    product_results = {}
    
    for method, description in methods_to_test:
        print(f"\\n🔄 {description} 테스트")
        print("-" * 50)
        
        # OCR의 경우 시간이 많이 걸리므로 다른 방법이 성공하면 건너뛰기
        if method == 'ocr':
            successful_text_methods = [m for m in ['pypdf2', 'pdfplumber'] 
                                     if m in product_results and product_results[m]['success'] 
                                     and len(product_results[m]['text']) > 100]
            
            if successful_text_methods:
                print(f"   ⏭️ 다른 방법이 성공했으므로 OCR을 건너뜁니다")
                continue
        
        result = safe_extract_text_from_pdf(product_pdf_path, method=method)
        product_results[method] = result
        
        if result['success']:
            # 텍스트 저장
            saved_file = simple_save_text(
                result['text'], 
                product_pdf_path.stem, 
                result['method']
            )
            result['saved_file'] = saved_file
            
            # 텍스트 미리보기
            if len(result['text']) > 0:
                preview = result['text'][:200].replace('\\n', ' ').strip()
                print(f"   👀 미리보기: {preview}...")
                
                # 텍스트 품질 분석
                korean_chars = len([c for c in result['text'] if '가' <= c <= '힣'])
                total_chars = len(result['text'])
                if total_chars > 0:
                    korean_ratio = korean_chars / total_chars * 100
                    print(f"   🇰🇷 한글 비율: {korean_ratio:.1f}%")
            else:
                print(f"   ⚠️ 추출된 텍스트가 없습니다")
        
        # 첫 번째 성공한 방법이 충분한 텍스트를 추출했다면 다음 방법은 건너뛰기
        if result['success'] and len(result['text']) > 1000:
            print(f"   ✨ 충분한 텍스트가 추출되었습니다. 다음 방법들을 건너뜁니다.")
            break
    
    # 결과 비교 및 요약
    print(f"\\n📊 ProductSpec PDF 테스트 결과 요약")
    print("=" * 80)
    
    successful_product_methods = []
    
    for method, description in methods_to_test:
        if method in product_results:
            result = product_results[method]
            
            if result['success']:
                text_len = result.get('text_length', len(result.get('text', '')))
                proc_time = result.get('processing_time', 0)
                
                print(f"✅ {description}")
                print(f"   📝 추출 텍스트: {text_len:,}자")
                print(f"   ⏱️ 처리 시간: {proc_time:.2f}초")
                print(f"   📄 페이지 수: {result.get('pages', 'N/A')}")
                
                successful_product_methods.append((method, text_len, proc_time))
                
                if 'saved_file' in result and result['saved_file']:
                    print(f"   💾 저장됨: {result['saved_file'].name}")
                
                # 텍스트 내용 분석
                text_content = result.get('text', '')
                if text_content:
                    # 주요 키워드 검색
                    keywords = ['insulin', '인슐린', 'pump', '펌프', 'smart', '스마트', 'glucose', '혈당']
                    found_keywords = [kw for kw in keywords if kw.lower() in text_content.lower()]
                    if found_keywords:
                        print(f"   🔍 발견된 키워드: {', '.join(found_keywords)}")
                
            else:
                print(f"❌ {description}")
                print(f"   🚫 오류: {result.get('error', 'Unknown error')}")
            
            print()
    
    # 최적 방법 추천
    if successful_product_methods:
        print(f"🏆 ProductSpec PDF 최적 방법 추천")
        print("-" * 50)
        
        # 가장 많은 텍스트를 추출한 방법
        best_quality = max(successful_product_methods, key=lambda x: x[1])
        print(f"🎯 최고 품질: {best_quality[0]} ({best_quality[1]:,}자)")
        
        # 가장 빠른 방법
        best_speed = min(successful_product_methods, key=lambda x: x[2])
        print(f"⚡ 최고 속도: {best_speed[0]} ({best_speed[2]:.2f}초)")
        
        # 일반 PDF 특성에 맞는 추천
        if best_quality[1] > 1000:
            print(f"💡 추천: {best_quality[0]} (일반 PDF - 텍스트 추출 우수)")
        else:
            print(f"💡 추천: OCR 방법을 시도해보세요 (텍스트 기반 추출 실패)")
            
    else:
        print(f"❌ 모든 방법이 실패했습니다.")
        print(f"💡 다음을 확인해보세요:")
        print(f"   - 파일이 손상되지 않았는지")
        print(f"   - PDF 파일이 암호화되어 있지 않은지") 
        print(f"   - OCR 방법을 시도해보세요")
    
    # 이전 test.pdf와 비교
    if 'results' in globals() and results:
        print(f"\\n📊 파일별 비교 분석")
        print("=" * 50)
        print(f"🔬 test.pdf (이미지 기반):")
        print(f"   - 최적 방법: OCR")
        print(f"   - 추출량: 84,097자")
        print(f"   - 처리시간: ~6.5분")
        
        print(f"\\n📋 ProductSpec PDF (일반 PDF):")
        if successful_product_methods:
            best_method = max(successful_product_methods, key=lambda x: x[1])
            print(f"   - 최적 방법: {best_method[0]}")
            print(f"   - 추출량: {best_method[1]:,}자")
            print(f"   - 처리시간: {best_method[2]:.2f}초")
        else:
            print(f"   - 추출 실패")

else:
    print(f"❌ 테스트 파일이 없습니다: {product_pdf_path}")
    
    # 대안 파일 제안
    input_dir = product_pdf_path.parent
    if input_dir.exists():
        pdf_files = list(input_dir.glob("*ProductSpec*.pdf"))
        if not pdf_files:
            pdf_files = list(input_dir.glob("*.pdf"))
        
        if pdf_files:
            print(f"\\n📂 사용 가능한 PDF 파일들:")
            for i, pdf_file in enumerate(pdf_files[:10], 1):
                size_mb = pdf_file.stat().st_size / 1024 / 1024
                print(f"   {i}. {pdf_file.name} ({size_mb:.2f} MB)")

print(f"\\n🎯 일반 PDF 테스트 완료!")
print(f"📁 추출된 파일들은 다음 경로에 저장됩니다:")
print(f"   /home/admin/wkms-aws/jupyter_notebook/data/opensource_output/extracted_texts/")

In [ ]:
# 📊 test.pdf 추출 방법별 상세 비교 테스트

# test.pdf가 존재하는 경우에만 실행
if test_pdf_path.exists():
    print("🔬 test.pdf 추출 방법별 상세 비교")
    print("=" * 80)
    
    extraction_methods = [
        ('auto', '자동 선택'),
        ('pypdf2', 'PyPDF2'),
        ('pdfplumber', 'pdfplumber'),
        ('ocr', 'OCR (Tesseract)')
    ]
    
    results = {}
    
    for method, description in extraction_methods:
        print(f"\n🔄 {description} 방법 테스트 중...")
        try:
            result = text_extractor.extract_text(str(test_pdf_path), method=method)
            
            if result['success']:
                text_length = len(result['text'])
                processing_time = result['processing_time']
                
                results[method] = {
                    'success': True,
                    'text_length': text_length,
                    'processing_time': processing_time,
                    'extraction_method': result['extraction_method'],
                    'metadata': result.get('metadata', {})
                }
                
                print(f"   ✅ 성공: {text_length:,}자, {processing_time:.2f}초")
                
                # 첫 100자 미리보기
                preview = result['text'][:100].replace('\n', ' ')
                print(f"   👀 미리보기: {preview}...")
                
            else:
                results[method] = {
                    'success': False,
                    'error': result.get('error', 'Unknown error')
                }
                print(f"   ❌ 실패: {result.get('error', 'Unknown error')}")
                
        except Exception as e:
            results[method] = {
                'success': False,
                'error': str(e)
            }
            print(f"   ❌ 오류: {e}")
    
    # 결과 비교표 출력
    print(f"\n📈 추출 방법별 결과 비교")
    print("-" * 80)
    print(f"{'방법':<15} {'성공':<6} {'텍스트 길이':<12} {'처리시간':<10} {'실제 방법'}")
    print("-" * 80)
    
    for method, description in extraction_methods:
        if method in results:
            result = results[method]
            if result['success']:
                success_icon = "✅"
                text_len = f"{result['text_length']:,}자"
                proc_time = f"{result['processing_time']:.2f}초"
                actual_method = result['extraction_method']
            else:
                success_icon = "❌"
                text_len = "N/A"
                proc_time = "N/A"
                actual_method = "실패"
            
            print(f"{description:<15} {success_icon:<6} {text_len:<12} {proc_time:<10} {actual_method}")
    
    # 최적 방법 추천
    successful_results = {k: v for k, v in results.items() if v['success']}
    if successful_results:
        # 텍스트 길이 기준으로 최고 성능 찾기
        best_method = max(successful_results.items(), key=lambda x: x[1]['text_length'])
        
        print(f"\n🏆 추천 방법: {best_method[0]} ({best_method[1]['text_length']:,}자)")
        
        # 속도 vs 품질 분석
        speed_winner = min(successful_results.items(), key=lambda x: x[1]['processing_time'])
        quality_winner = max(successful_results.items(), key=lambda x: x[1]['text_length'])
        
        print(f"\n⚡ 최고 속도: {speed_winner[0]} ({speed_winner[1]['processing_time']:.2f}초)")
        print(f"🎯 최고 품질: {quality_winner[0]} ({quality_winner[1]['text_length']:,}자)")
        
        if speed_winner[0] != quality_winner[0]:
            speed_text_ratio = quality_winner[1]['text_length'] / speed_winner[1]['text_length']
            time_ratio = quality_winner[1]['processing_time'] / speed_winner[1]['processing_time']
            print(f"📊 품질 개선율: {speed_text_ratio:.1f}배, 시간 증가율: {time_ratio:.1f}배")
        
    else:
        print(f"\n❌ 모든 추출 방법이 실패했습니다.")

else:
    print("⚠️ test.pdf 파일이 없어 비교 테스트를 건너뜁니다.")

## 🎯 오픈소스 문서 처리 파이프라인 완료!

### ✅ **구현된 기능**
1. **📚 라이브러리 로딩**: 15개 오픈소스 라이브러리 통합
2. **⚙️ 설정 관리**: 디렉토리 구조 및 처리 옵션 설정
3. **📂 문서 로딩**: 7가지 파일 형식 지원 (PDF, Word, PPT, Excel, HTML, 텍스트, 이미지)
4. **🔤 텍스트 추출**: 각 형식별 최적화된 추출 방법
5. **🔍 OCR 처리**: Tesseract 기반 이미지 PDF 처리 (페이지별 로그 및 저장)
6. **📊 전처리**: 텍스트 정제 및 청킹
7. **💾 결과 저장**: 추출된 텍스트 파일 저장

### 🔗 **다음 단계 연결**
- **백엔드 연동**: 추출된 텍스트를 `tools.py`의 `chunked_texts()` 함수로 전달
- **임베딩 생성**: Azure OpenAI를 통한 벡터 임베딩
- **벡터스토어**: Cosmos DB 또는 Azure AI Search에 저장

### ? **출력 파일 구조**
```
jupyter_notebook/data/opensource_output/
├── extracted_texts/           # 추출된 텍스트 파일
│   ├── 파일명_extracted.txt   # 일반 추출 결과
│   ├── 파일명_page_001.txt   # OCR 페이지별 결과
│   └── 파일명_ocr_full.txt   # OCR 전체 결과
└── metadata/                  # 처리 메타데이터
    └── 파일명_ocr_metadata.json
```

### 💡 **사용법 요약**
```python
# 개별 파일 테스트
test_result = test_file_extraction("문서.pdf")

# OCR 강제 사용 (이미지 기반 PDF)
test_result = test_file_extraction("이미지.pdf", method='ocr')

# 배치 테스트
for file_path in sample_files:
    test_file_extraction(file_path)
```

**🎉 이제 다양한 문서 형식에서 텍스트를 추출하고 저장할 수 있습니다!**